In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy.stats import entropy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/Libin's Project/")
print(os.getcwd())

/content
/content/drive/My Drive/Libin's Project


In [4]:
class Model(): #tf.Module
  def __init__(self, name=None):

    self.name = name
    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):

    self.model =  tf.keras.Sequential()

    self.model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',strides=1, input_shape=(28,28,1), use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
    self.model.add(tf.keras.layers.Flatten())
    self.model.add(tf.keras.layers.Dense(128))
    self.model.add(tf.keras.layers.Dense(8)) #, activation='softmax'))
    self.model.add(Activation('softmax'))
    return self.model

  def apply_softmax(self, logits):
    return tf.nn.softmax(logits)

  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

      with tf.GradientTape() as tape:
          logits = self.model(img_batch)
#          logits = self.apply_softmax(logits)
          xent = self.loss_fn(lbl_batch, logits)

      varis = self.model.trainable_variables
      grads = tape.gradient(xent, varis)

      self.opt.apply_gradients(zip(grads, varis))
      
      self.train_acc_metric(lbl_batch, logits)
      
      if tf.equal(step % 100, 0): #not step % 500:
        acc = self.train_acc_metric.result()
        tf.print("Loss: {} Accuracy: {}".format(xent, acc))
        self.train_acc_metric.reset_states()

  def train2(self, train_imgs, train_lbls, val_imgs, val_lbls):
    self.model.compile(
      optimizer='adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['accuracy']
      )

    self.model.fit(train_imgs, train_lbls, validation_data= (val_imgs, val_lbls), epochs=20)

  def test(self,test_data, temp=None):
    for img_batch, lbl_batch in test_data:
      logits = self.model(img_batch)
      if not temp is None:
        logits = tf.math.divide(logits, temp)
      pred = self.apply_softmax(logits)
      self.test_acc_metric(lbl_batch, pred)
    tf.print("Test acc: {}".format(self.test_acc_metric.result()))

  def save_model(self, path):
    self.model.save(path)

  def load_model(self, path):
    self.model = tf.keras.models.load_model(path)
    return self.model

  def get_max_pred_value(self, data, temp=None):
    logits = self.model(data)
    if not temp is None:
      logits = tf.math.divide(logits, temp)
    pred = self.apply_softmax(logits)
    return np.max(pred)


In [5]:
class BaselineModel(): #tf.Module
  def __init__(self, name=None):

    self.name = name
    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):

    self.model =  tf.keras.Sequential()

    self.model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',strides=1, input_shape=(28,28,1), use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
    self.model.add(tf.keras.layers.Flatten())
    self.model.add(tf.keras.layers.Dense(128))
    self.model.add(tf.keras.layers.Dense(9)) #, activation='softmax'))
    self.model.add(Activation('softmax'))
    return self.model

  def apply_softmax(self, logits):
    return tf.nn.softmax(logits)

  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

      with tf.GradientTape() as tape:
          logits = self.model(img_batch)
#          logits = self.apply_softmax(logits)
          xent = self.loss_fn(lbl_batch, logits)

      varis = self.model.trainable_variables
      grads = tape.gradient(xent, varis)

      self.opt.apply_gradients(zip(grads, varis))
      
      self.train_acc_metric(lbl_batch, logits)
      
      if tf.equal(step % 100, 0): #not step % 500:
        acc = self.train_acc_metric.result()
        tf.print("Loss: {} Accuracy: {}".format(xent, acc))
        self.train_acc_metric.reset_states()

  def train2(self, train_imgs, train_lbls, val_imgs, val_lbls):
    self.model.compile(
      optimizer='adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['accuracy']
      )

    self.model.fit(train_imgs, train_lbls, validation_data= (val_imgs, val_lbls), epochs=20)

  def test(self,test_data, temp=None):
    for img_batch, lbl_batch in test_data:
      logits = self.model(img_batch)
      if not temp is None:
        logits = tf.math.divide(logits, temp)
      pred = self.apply_softmax(logits)
      self.test_acc_metric(lbl_batch, pred)
    tf.print("Test acc: {}".format(self.test_acc_metric.result()))

  def save_model(self, path):
    self.model.save(path)

  def load_model(self, path):
    self.model = tf.keras.models.load_model(path)
    return self.model

  def get_max_pred_value(self, data, temp=None):
    logits = self.model(data)
    if not temp is None:
      logits = tf.math.divide(logits, temp)
    pred = self.apply_softmax(logits)
    return np.max(pred)


In [ ]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()
train_imgs = (train_imgs.astype(np.float32) / 255.).reshape((-1, 28, 28, 1))
train_lbls = train_lbls.astype(np.int32)


test_imgs = (test_imgs.astype(np.float32) / 255.).reshape((-1, 28, 28, 1))
test_lbls = test_lbls.astype(np.int32)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(train_imgs, train_lbls, test_size=0.20, random_state=42)

In [ ]:
pickle_out = open("mnist_data","wb")
pickle.dump([train_images, train_labels, val_images, val_labels, test_imgs, test_lbls], pickle_out)
pickle_out.close()

In [6]:
pickle_in = open("mnist_data","rb")
train_images, train_labels, val_images, val_labels, test_imgs, test_lbls = pickle.load(pickle_in)

In [7]:
def separate_class(class_val, train_images, train_labels, val_images, val_labels):
  new_train_images = []
  new_train_labels = []
  for ind, im in enumerate(train_images):
    if train_labels[ind] != class_val:
      new_train_images.append(im)
      new_train_labels.append(train_labels[ind])

  new_train_images = np.asarray(new_train_images)
  new_train_labels = np.asarray(new_train_labels)

  new_val_images = []
  new_val_labels = []
  for ind, im in enumerate(val_images):
    if val_labels[ind] != class_val:
      new_val_images.append(im)
      new_val_labels.append(val_labels[ind])

  new_val_images = np.asarray(new_val_images)
  new_val_labels = np.asarray(new_val_labels)

  return new_train_images, new_train_labels, new_val_images, new_val_labels

In [ ]:
trial = set(new_train_labels)
trial.remove(1)
print(trial)
trial = sorted(trial)
print(trial)
mapping = {}
for i,j in enumerate(trial):
  mapping[j] = i
print(mapping)

In [8]:
def get_train_data(leave_out_class, train_images, train_labels):
  class_train_images = []
  class_train_labels = []
  mapping = {}
  
  labels = set(train_labels)
  labels.remove(leave_out_class)
  labels = sorted(labels)
  for i,j in enumerate(labels):
    mapping[j] = i
  print(mapping)

  for ind, im in enumerate(train_images):
    if train_labels[ind] != leave_out_class:
      class_train_images.append(im)
      class_train_labels.append(mapping[train_labels[ind]]) #(train_labels[ind])


  class_train_images = np.asarray(class_train_images)
  class_train_labels = np.asarray(class_train_labels)

  print(set(class_train_labels))
  print(class_train_labels.shape)

  train_data = tf.data.Dataset.from_tensor_slices((class_train_images, class_train_labels))

  train_data = train_data.shuffle(class_train_images.shape[0])
  train_data = train_data.batch(128)
  train_data = train_data.repeat(20)

  return class_train_images, class_train_labels # train_data

In [9]:
def temp_cal(y_pred, val_labels):

  temp = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32) 

  def compute_loss():
      y_pred_model_w_temp = tf.math.divide(y_pred, temp)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                  tf.convert_to_tensor(keras.utils.to_categorical(val_labels)), y_pred_model_w_temp))
      return loss

  optimizer = tf.optimizers.Adam(learning_rate=0.01)

  print('Temperature Initial value: {}'.format(temp.numpy()))

  for i in range(300):
      opts = optimizer.minimize(compute_loss, var_list=[temp])


  print('Temperature Final value: {}'.format(temp.numpy()))

  return temp

In [10]:
def temp_scaling(y_pred, temp):
  return tf.math.divide(y_pred, temp)

# Baseline

In [ ]:
model0 = BaselineModel(name='baseline:anomaly0')
model1 = BaselineModel(name='baseline:anomaly1')
model2 = BaselineModel(name='baseline:anomaly2')
model3 = BaselineModel(name='baseline:anomaly3')
model4 = BaselineModel(name='baseline:anomaly4')
model5 = BaselineModel(name='baseline:anomaly5')
model6 = BaselineModel(name='baseline:anomaly6')
model7 = BaselineModel(name='baseline:anomaly7')
model8 = BaselineModel(name='baseline:anomaly8')
model9 = BaselineModel(name='baseline:anomaly9')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()
class_9 = model9.build()

In [ ]:
set(train_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:

train_imgs0, train_lbls0 = get_train_data(0, train_images, train_labels)
val_imgs0, val_lbls0 = get_train_data(0, val_images, val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/baseline-anomaly0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, train_images, train_labels)
val_imgs1, val_lbls1 = get_train_data(1, val_images, val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/baseline-anomaly1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, train_images, train_labels)
val_imgs2, val_lbls2 = get_train_data(2, val_images, val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/baseline-anomaly2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, train_images, train_labels)
val_imgs3, val_lbls3 = get_train_data(3, val_images, val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/baseline-anomaly3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, train_images, train_labels)
val_imgs4, val_lbls4 = get_train_data(4, val_images, val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/baseline-anomaly4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(5, train_images, train_labels)
val_imgs5, val_lbls5 = get_train_data(5, val_images, val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/baseline-anomaly5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(6, train_images, train_labels)
val_imgs6, val_lbls6 = get_train_data(6, val_images, val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/baseline-anomaly6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(7, train_images, train_labels)
val_imgs7, val_lbls7 = get_train_data(7, val_images, val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/baseline-anomaly7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(8, train_images, train_labels)
val_imgs8, val_lbls8 = get_train_data(8, val_images, val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/baseline-anomaly8")
print('Model 8 saved')

train_imgs9, train_lbls9 = get_train_data(9, train_images, train_labels)
val_imgs9, val_lbls9 = get_train_data(9, val_images, val_labels)
class_9 = model9.build()
model9.train2(train_imgs9, train_lbls9, val_imgs9, val_lbls9)
#model9.train(train_data9)
model9.save_model("final_models/baseline-anomaly9")
print('Model 9 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8}
{0, 1, 2, 3, 4, 5, 6, 7, 8}
(43252,)
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8}
{0, 1, 2, 3, 4, 5, 6, 7, 8}
(10825,)
Epoch 1/20
1352/1352 [==============================] - 45s 33ms/step - loss: 5.5498 - accuracy: 0.7934 - val_loss: 0.1134 - val_accuracy: 0.9708
Epoch 2/20
1352/1352 [==============================] - 44s 32ms/step - loss: 0.1002 - accuracy: 0.9734 - val_loss: 0.0683 - val_accuracy: 0.9805
Epoch 3/20
1352/1352 [==============================] - 44s 32ms/step - loss: 0.0720 - accuracy: 0.9786 - val_loss: 0.1099 - val_accuracy: 0.9719
Epoch 4/20
1352/1352 [==============================] - 44s 32ms/step - loss: 0.0701 - accuracy: 0.9786 - val_loss: 0.5159 - val_accuracy: 0.9167
Epoch 5/20
1352/1352 [==============================] - 44s 32ms/step - loss: 0.0660 - accuracy: 0.9802 - val_loss: 0.1031 - val_accuracy: 0.9752
Epoch 6/20
1352/1352 [==============================] - 44s 32ms/step - loss: 0.0577 -

In [ ]:
class_0 = model0.load_model("final_models/baseline-anomaly0")
class_1 = model1.load_model("final_models/baseline-anomaly1")
class_2 = model2.load_model("final_models/baseline-anomaly2")
class_3 = model3.load_model("final_models/baseline-anomaly3")
class_4 = model4.load_model("final_models/baseline-anomaly4")
class_5 = model5.load_model("final_models/baseline-anomaly5")
class_6 = model6.load_model("final_models/baseline-anomaly6")
class_7 = model7.load_model("final_models/baseline-anomaly7")
class_8 = model8.load_model("final_models/baseline-anomaly8")
class_9 = model9.load_model("final_models/baseline-anomaly9")

In [ ]:
def get_max_pred_value(model, img):
  pred = model(img)

#  logits = tf.math.divide(logits, temp)
#  pred = tf.nn.softmax(logits)
#  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy()

In [ ]:
def get_mapping(anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
def get_bl_sm(model, anomaly_class):
  max_sm_all_bl = []
  y_true = []
  for data, label in tqdm(zip(test_imgs, test_lbls)):
    img = data.reshape([-1, 28, 28, 1])
    if label == anomaly_class:
      lbl = ood
    else:
      lbl = ind
    y_true.append(lbl)
    max0, pred_0 = get_max_pred_value(model, img)
    map0 = get_mapping(anomaly_class)
    pred_0 = map0[pred_0]

    max_sm_all_bl.append(max0)

  return y_true, max_sm_all_bl

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
print('For 0 as anomaly')
y_true, y_sm = get_bl_sm(class_0, 0)
a0 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a0)

In [ ]:
y_temp = []
for i in y_true:
  if i == 0:
    lbl = ood
  else:
    lbl = ind
  y_temp.append(lbl)
roc_auc_score(y_temp, y_sm)

0.8521517941988326

In [ ]:
for i, j in zip(y_temp, y_sm):
  print(i,j)

Streaming output truncated to the last 5000 lines.
1 0.99999976
1 0.9999715
1 1.0
1 1.0
1 0.99999917
1 0.9999999
0 0.9999795
1 1.0
0 0.99959475
1 0.9999987
1 1.0
1 1.0
1 1.0
1 0.9999999
1 1.0
1 0.9999995
1 1.0
1 0.9999981
1 1.0
1 1.0
1 1.0
1 1.0
1 0.99999976
1 1.0
1 1.0
1 1.0
1 0.9999999
1 0.9919872
1 1.0
1 0.9999994
1 1.0
0 0.91993225
1 1.0
0 0.8818021
1 1.0
0 1.0
1 1.0
1 1.0
1 1.0
1 1.0
1 0.99999845
1 1.0
1 1.0
0 0.9999995
1 0.9999999
1 1.0
1 0.95992136
1 1.0
1 0.9999908
1 1.0
1 1.0
0 0.9999993
1 1.0
1 1.0
1 1.0
1 1.0
1 0.99999785
1 1.0
1 0.99999905
1 1.0
1 0.9999944
1 1.0
1 0.9999993
1 1.0
1 1.0
1 0.99999976
1 1.0
1 0.6005787
1 0.87515426
0 0.9979354
1 1.0
1 1.0
1 1.0
1 1.0
1 0.9999939
1 1.0
1 1.0
1 0.9999999
1 0.99950373
0 0.8030544
1 1.0
1 0.99999964
1 0.9999807
1 0.9981358
1 0.99999595
1 1.0
1 1.0
0 0.60002834
1 1.0
1 1.0
1 0.999995
1 0.99999297
1 0.99999964
1 0.9999999
1 1.0
1 1.0
1 0.9999999
1 1.0
1 1.0
1 1.0
1 0.9999932
1 0.9999994
1 1.0
1 1.0
1 1.0
1 0.99995637
1 0.99999666
1

In [ ]:
print('For 0 as anomaly')
y_true, y_sm = get_bl_sm(class_0, 0)
a0 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a0)

print('-------------------')

print('For 1 as anomaly')
y_true, y_sm = get_bl_sm(class_1, 1)
a1 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a1)

print('-------------------')

print('For 2 as anomaly')
y_true, y_sm = get_bl_sm(class_2, 2)
a2 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a2)

print('-------------------')

print('For 3 as anomaly')
y_true, y_sm = get_bl_sm(class_3, 3)
a3 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a3)

print('-------------------')

print('For 4 as anomaly')
y_true, y_sm = get_bl_sm(class_4, 4)
a4 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a4)

print('-------------------')

print('For 5 as anomaly')
y_true, y_sm = get_bl_sm(class_5, 5)
a5 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a5)

print('-------------------')

print('For 6 as anomaly')
y_true, y_sm = get_bl_sm(class_6, 6)
a6 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a6)

print('-------------------')

print('For 7 as anomaly')
y_true, y_sm = get_bl_sm(class_7, 7)
a7 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a7)

print('-------------------')

print('For 8 as anomaly')
y_true, y_sm = get_bl_sm(class_8, 8)
a8 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a8)

print('-------------------')

print('For 9 as anomaly')
y_true, y_sm = get_bl_sm(class_9, 9)
a9 = roc_auc_score(y_true, y_sm)
print('AUROC: ', a9)

7it [00:00, 66.49it/s]

For 0 as anomaly


10000it [02:18, 72.43it/s]
8it [00:00, 74.27it/s]

AUROC:  0.8521517941988326
-------------------
For 1 as anomaly


10000it [02:18, 71.99it/s]
7it [00:00, 68.48it/s]

...

AUROC:  0.8219270263214047
-------------------
For 8 as anomaly


10000it [02:17, 72.50it/s]
8it [00:00, 75.08it/s]

AUROC:  0.7536742474739868
-------------------
For 9 as anomaly


10000it [02:18, 72.23it/s]

AUROC:  0.9370816692697543


In [ ]:
len(y_true), len(y_sm)

(10000, 10000)

In [ ]:
pickle_out = open("sm_all_wt_0an.pickle","wb")
pickle.dump(max_sm_all_wt_0an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_0an.pickle","rb")
max_sm_all_wt_0an = pickle.load(pickle_in)

# 0 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(0, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='0anomaly:classifier0')
model1 = Model(name='0anomaly:classifier1')
model2 = Model(name='0anomaly:classifier2')
model3 = Model(name='0anomaly:classifier3')
model4 = Model(name='0anomaly:classifier4')
model5 = Model(name='0anomaly:classifier5')
model6 = Model(name='0anomaly:classifier6')
model7 = Model(name='0anomaly:classifier7')
model8 = Model(name='0anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
set(new_train_labels)

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
train_imgs0, train_lbls0 = get_train_data(1, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(1, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/0anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(2, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(2, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/0anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(3, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(3, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/0anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(4, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(4, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/0anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(5, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(5, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/0anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/0anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/0anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/0anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/0anomaly-classifier8")
print('Model 8 saved')

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(37832,)
{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9503,)
Epoch 1/20
1183/1183 [==============================] - 37s 31ms/step - loss: 5.3148 - accuracy: 0.7415 - val_loss: 0.7003 - val_accuracy: 0.9368
Epoch 2/20
1183/1183 [==============================] - 36s 31ms/step - loss: 0.1938 - accuracy: 0.9551 - val_loss: 1.0786 - val_accuracy: 0.9144
Epoch 3/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.1143 - accuracy: 0.9708 - val_loss: 0.1462 - val_accuracy: 0.9636
Epoch 4/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0951 - accuracy: 0.9747 - val_loss: 0.2603 - val_accuracy: 0.9512
Epoch 5/20
1183/1183 [==============================] - 37s 32ms/step - loss: 0.0732 - accuracy: 0.9776 - val_loss: 0.0987 - val_accuracy: 0.9764
Epoch 6/20
1183/1183 [==============================] - 38s 32ms/step - loss: 0.0584 - accuracy: 0.9822 -

In [ ]:
class_0 = model0.load_model("final_models/0anomaly-classifier0")
class_1 = model1.load_model("final_models/0anomaly-classifier1")
class_2 = model2.load_model("final_models/0anomaly-classifier2")
class_3 = model3.load_model("final_models/0anomaly-classifier3")
class_4 = model4.load_model("final_models/0anomaly-classifier4")
class_5 = model5.load_model("final_models/0anomaly-classifier5")
class_6 = model6.load_model("final_models/0anomaly-classifier6")
class_7 = model7.load_model("final_models/0anomaly-classifier7")
class_8 = model8.load_model("final_models/0anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35', 'dense_37', 'dense_39', 'dense_41']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9503,)
Temperature Initial value: 1.0
Temperature Final value: 2.29171085357666
{1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9651,)
Temperature Initial value: 1.0
Temperature Final value: 1.7892318964004517
{1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9606,)
Temperature Initial value: 1.0
Temperature Final value: 2.168271780014038
{1: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9649,)
Temperature Initial value: 1.0
Temperature Final value: 2.192014694213867
{1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9721,)
Temperature Initial value: 1.0
Temperature Final value: 2.3539624214172363
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9648,)
Temperature Initial value: 1.0
Temperature Final value: 1.8671849966049194
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2917109>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.7892319>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1682718>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1920147>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3539624>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.867185>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2140534>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.8692174>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.201385>]

In [ ]:
temp_val = [2.2917109, 1.7892319, 2.1682718, 2.1920147, 2.3539624, 1.867185, 2.2140534, 1.8692174, 2.201385]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 1:
    logits = model_dict[0](img)
    pred = tf.nn.softmax(logits)
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 2:
    logits = model_dict[1](img)
    pred = tf.nn.softmax(logits)
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 3:
    logits = model_dict[2](img)
    pred = tf.nn.softmax(logits)
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 4:
    logits = model_dict[3](img)
    pred = tf.nn.softmax(logits)
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    pred = tf.nn.softmax(logits)
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    pred = tf.nn.softmax(logits)
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    pred = tf.nn.softmax(logits)
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    pred = tf.nn.softmax(logits)
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    pred = tf.nn.softmax(logits)
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)

classifier_avg = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg[i]

treshold = treshold_value/len(classifier_avg)

print('Threshold:', treshold)

In [ ]:
set(new_val_labels)

{1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 1:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 2:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

Streaming output truncated to the last 5000 lines.
826it [01:01, 13.73it/s]
828it [01:01, 13.85it/s]
830it [01:02, 14.07it/s]
832it [01:02, 14.12it/s]
834it [01:02, 13.30it/s]
836it [01:02, 12.95it/s]
838it [01:02, 12.67it/s]
840it [01:02, 13.07it/s]
842it [01:02, 13.19it/s]
844it [01:03, 13.46it/s]
846it [01:03, 13.78it/s]
848it [01:03, 14.00it/s]
850it [01:03, 13.91it/s]
852it [01:03, 13.99it/s]
854it [01:03, 13.79it/s]
856it [01:04, 13.63it/s]
858it [01:04, 13.88it/s]
860it [01:04, 13.75it/s]
862it [01:04, 13.96it/s]
864it [01:04, 14.14it/s]
866it [01:04, 14.05it/s]
868it [01:04, 14.02it/s]
870it [01:04, 14.04it/s]
872it [01:05, 14.19it/s]
874it [01:05, 14.03it/s]
876it [01:05, 13.80it/s]
878it [01:05, 13.26it/s]
880it [01:05, 12.80it/s]
882it [01:05, 13.07it/s]
884it [01:06, 13.18it/s]
886it [01:06, 13.36it/s]
888it [01:06, 13.29it/s]
890it [01:06, 13.59it/s]
892it [01:06, 13.76it/s]
894it [01:06, 13.83it/s]
896it [01:06, 13.65it/s]
898it [01:07, 13.89it/s]
900it [01:07, 14.00it/s]

[9357.497582048178, 9446.468951970339, 9378.525211721659, 9479.95735681057, 9535.449575036764, 9510.309046983719, 9289.561259806156, 9530.35733640194, 9517.998938679695]
[9503, 9651, 9606, 9649, 9721, 9648, 9526, 9665, 9631]
[448.41239222848185, 637.2631376257617, 684.1313679411988, 512.6794190745065, 558.3709353026475, 448.10474678606613, 734.9538882930817, 437.2487093207777, 359.9448776579988]
Entropy: 0.05569118810380174
Each Classifier Average:  [0.9846887911236639, 0.9788072688809801, 0.9763195098606765, 0.9824808121888869, 0.9809124138500941, 0.9857285496459078, 0.9751796409622251, 0.9860690467048049, 0.988266944105461]
Threshold: 0.9820503308136335


In [ ]:
ref_vect_in_0an = []
ref_vect_in_0an.append(classifier_avg_t)

threshold_in_0an = []
threshold_in_0an.append(treshold_t)

entropy_in_0an = []
entropy_in_0an.append(entropy_t)

print(entropy_in_0an)
print(ref_vect_in_0an)
print(threshold_in_0an)

[0.05569118810380174]
[[0.9846887911236639, 0.9788072688809801, 0.9763195098606765, 0.9824808121888869, 0.9809124138500941, 0.9857285496459078, 0.9751796409622251, 0.9860690467048049, 0.988266944105461]]
[0.9820503308136335]


In [ ]:
entropy_in_0an = [0.05569118810380174]
ref_vect_in_0an = [[0.9846887911236639, 0.9788072688809801, 0.9763195098606765, 0.9824808121888869, 0.9809124138500941, 0.9857285496459078, 0.9751796409622251, 0.9860690467048049, 0.988266944105461]]
threshold_in_0an = [0.9820503308136335]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 1:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 2:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)


0it [00:00, ?it/s]
12it [00:00, 116.56it/s]
24it [00:00, 114.83it/s]
36it [00:00, 113.79it/s]
48it [00:00, 112.99it/s]
60it [00:00, 112.31it/s]
72it [00:00, 112.22it/s]
83it [00:00, 111.04it/s]
95it [00:00, 110.92it/s]
106it [00:00, 108.37it/s]
117it [00:01, 107.39it/s]
128it [00:01, 105.91it/s]
139it [00:01, 106.60it/s]
150it [00:01, 107.46it/s]
162it [00:01, 109.86it/s]
174it [00:01, 110.56it/s]
186it [00:01, 108.11it/s]
197it [00:01, 104.92it/s]
208it [00:01, 101.83it/s]
219it [00:02, 102.39it/s]
230it [00:02, 103.54it/s]
242it [00:02, 105.64it/s]
253it [00:02, 106.37it/s]
264it [00:02, 104.32it/s]
275it [00:02, 98.84it/s] 
285it [00:02, 97.50it/s]
295it [00:02, 97.03it/s]
305it [00:02, 96.00it/s]
315it [00:03, 94.17it/s]
325it [00:03, 93.97it/s]
336it [00:03, 96.80it/s]
347it [00:03, 99.96it/s]
359it [00:03, 103.11it/s]
370it [00:03, 104.37it/s]
382it [00:03, 106.45it/s]
393it [00:03, 106.92it/s]
405it [00:03, 108.38it/s]
416it [00:03, 107.81it/s]
427it [00:04, 106.02it/s]
438it [

[714.6198810487986, 982.1086966097355, 928.6873873472214, 927.3719318807125, 879.4890236258507, 993.4708969295025, 926.6441331803799, 911.2026011049747, 1013.5440234839916]
[1322, 1174, 1219, 1176, 1104, 1177, 1299, 1160, 1194]
[1573.1453399970196, 509.90413363599646, 757.7741318449798, 622.4137706265319, 573.614500436941, 476.6119991002597, 972.0077286690939, 658.5512278623064, 487.1469287318632]
Entropy: 0.6026340201245685
Each Classifier Average:  [0.5405596679642954, 0.8365491453234544, 0.7618436319501406, 0.788581574728497, 0.7966386083567488, 0.8440704306962638, 0.7133519116092224, 0.7855194837111851, 0.8488643412763749]
Threshold: 0.768442088401798


In [ ]:
ref_vect_out_0an = []
ref_vect_out_0an.append(classifier_avg_t)

threshold_out_0an = []
threshold_out_0an.append(treshold_t)

entropy_out_0an = []
entropy_out_0an.append(entropy_t)

print(entropy_out_0an)
print(ref_vect_out_0an)
print(threshold_out_0an)

[0.6026340201245685]
[[0.5405596679642954, 0.8365491453234544, 0.7618436319501406, 0.788581574728497, 0.7966386083567488, 0.8440704306962638, 0.7133519116092224, 0.7855194837111851, 0.8488643412763749]]
[0.768442088401798]


In [ ]:
entropy_out_0an = [0.6026340201245685]
ref_vect_out_0an = [[0.5405596679642954, 0.8365491453234544, 0.7618436319501406, 0.788581574728497, 0.7966386083567488, 0.8440704306962638, 0.7133519116092224, 0.7855194837111851, 0.8488643412763749]]
threshold_out_0an = [0.768442088401798]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
set(train_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_0an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 0:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(1, 0)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(2, 0)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(3, 0)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(4, 0)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(5, 0)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 0)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 0)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 0)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 0)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_0an.append([ref_value, lbl, pred_value, label, e_])


Streaming output truncated to the last 5000 lines.
5001it [11:38,  6.85it/s]
5002it [11:38,  6.73it/s]
5003it [11:38,  6.47it/s]
5004it [11:38,  6.38it/s]
5005it [11:38,  6.36it/s]
5006it [11:39,  6.37it/s]
5007it [11:39,  6.16it/s]
5008it [11:39,  6.42it/s]
5009it [11:39,  6.63it/s]
5010it [11:39,  6.72it/s]
5011it [11:39,  6.59it/s]
5012it [11:39,  6.54it/s]
5013it [11:40,  6.49it/s]
5014it [11:40,  6.67it/s]
5015it [11:40,  6.76it/s]
5016it [11:40,  6.75it/s]
5017it [11:40,  6.91it/s]
5018it [11:40,  6.97it/s]
5019it [11:40,  7.02it/s]
5020it [11:41,  6.96it/s]
5021it [11:41,  6.87it/s]
5022it [11:41,  7.02it/s]
5023it [11:41,  7.10it/s]
5024it [11:41,  6.91it/s]
5025it [11:41,  6.80it/s]
5026it [11:42,  6.67it/s]
5027it [11:42,  6.77it/s]
5028it [11:42,  6.74it/s]
5029it [11:42,  6.83it/s]
5030it [11:42,  6.86it/s]
5031it [11:42,  6.89it/s]
5032it [11:42,  6.96it/s]
5033it [11:43,  7.08it/s]
5034it [11:43,  7.12it/s]
5035it [11:43,  7.05it/s]
5036it [11:43,  7.05it/s]
5037it [11:43

In [ ]:
pred_value, label

([6, 6, 6, 6, 6, 5, 6, 6, 6], 6)

In [ ]:
pickle_out = open("sm_all_wt_0an.pickle","wb")
pickle.dump(max_sm_all_wt_0an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_0an.pickle","rb")
max_sm_all_wt_0an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_0an:

  print(data, lbl, a_l)
  break

[0.9999738, 0.99999714, 0.9999778, 0.9999527, 0.9999902, 0.9999989, 0.690764, 0.99995613, 0.9999988] 1 7


In [ ]:
from math import *

def manhattan_distance(x,y):
  return sum(abs(a-b) for a,b in zip(x,y))

def euclidean_distance(x,y):
  return sqrt(sum(pow(a-b,2) for a, b in zip(x, y)))

In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_0an:

  temp = data.copy()
  temp.append(e_)

  temp_ref = ref_vect_out_0an[0].copy()
  temp_ref.append(entropy_out_0an[0])

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_0an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_0an[0]))

  dist_in_man = manhattan_distance(data, ref_vect_in_0an[0])
  dist_ood_man = manhattan_distance(data, ref_vect_out_0an[0])

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_0an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_0an[0]))

  dist_ood_temp = np.linalg.norm(np.array(temp) - np.array(temp_ref)) 
  sim_ood_temp = 1 / (1 + dist_ood_temp)

#  dist_ood_man2 = manhattan_distance(np.array([e_]), np.array(entropy_out_0an)) 
#  dist_in_man2 = manhattan_distance(np.array([e_]), np.array(entropy_in_0an))

  dist_in = (dist_in + dist_in_man) / 2
  dist_ood = (dist_ood + dist_ood_man) / 2

#  dist_in2 = dist_in2 + dist_in_man2
#  dist_ood2 = dist_ood2 + dist_ood_man2

#  print(dist_in, dist_in_man, dist_ood, dist_ood_man)
#  dist_in = dist_in + dist_in2
#  dist_ood = dist_ood + dist_ood2

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

#  print(sim_in, sim_in2, sim_ood, sim_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  #sim_in = sim_in / (sim_in + sim_ood)
  #sim_ood = sim_ood / (sim_in + sim_ood)

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)
  if sim_in >= sim_ood:
    y_ = ind
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)

  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1
  #break

print("Total Accuracy: ", accuracy/len(max_sm_all_wt_0an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9275
Accuracy of determining ID data:  0.9284922394678492
Accuracy of determining OOD data:  0.9183673469387755


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.03620299561066112

In [ ]:
roc_auc_score(y_true, y_pred)

0.07385175799809947

In [ ]:
roc_auc_score(y_true, sim_score_)

0.9695009955201593

# 1 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(1, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='1anomaly:classifier0')
model1 = Model(name='1anomaly:classifier1')
model2 = Model(name='1anomaly:classifier2')
model3 = Model(name='1anomaly:classifier3')
model4 = Model(name='1anomaly:classifier4')
model5 = Model(name='1anomaly:classifier5')
model6 = Model(name='1anomaly:classifier6')
model7 = Model(name='1anomaly:classifier7')
model8 = Model(name='1anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
set(new_train_labels)

{0, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/1anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(2, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(2, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/1anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(3, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(3, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/1anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(4, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(4, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/1anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(5, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(5, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/1anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/1anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/1anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/1anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/1anomaly-classifier8")
print('Model 8 saved')

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(37832,)
{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9503,)
Epoch 1/20
1183/1183 [==============================] - 37s 31ms/step - loss: 5.7010 - accuracy: 0.7775 - val_loss: 0.4129 - val_accuracy: 0.9289
Epoch 2/20
1183/1183 [==============================] - 36s 31ms/step - loss: 0.1970 - accuracy: 0.9609 - val_loss: 0.2405 - val_accuracy: 0.9524
Epoch 3/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0742 - accuracy: 0.9803 - val_loss: 0.0784 - val_accuracy: 0.9760
Epoch 4/20
1183/1183 [==============================] - 37s 32ms/step - loss: 0.0708 - accuracy: 0.9793 - val_loss: 0.0877 - val_accuracy: 0.9764
Epoch 5/20
1183/1183 [==============================] - 38s 32ms/step - loss: 0.0635 - accuracy: 0.9816 - val_loss: 0.0872 - val_accuracy: 0.9762
Epoch 6/20
1183/1183 [==============================] - 38s 32ms/step - loss: 0.0574 - accuracy: 0.9811 -

In [ ]:
#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/1anomaly-classifier8")
print('Model 8 saved')

{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(37825,)
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9484,)
Epoch 1/20
1183/1183 [==============================] - 43s 30ms/step - loss: 5.4658 - accuracy: 0.8050 - val_loss: 1.7318 - val_accuracy: 0.8135
Epoch 2/20
1183/1183 [==============================] - 36s 31ms/step - loss: 0.6752 - accuracy: 0.9087 - val_loss: 0.3964 - val_accuracy: 0.9406
Epoch 3/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0844 - accuracy: 0.9768 - val_loss: 0.0801 - val_accuracy: 0.9766
Epoch 4/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0587 - accuracy: 0.9837 - val_loss: 0.0883 - val_accuracy: 0.9778
Epoch 5/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0557 - accuracy: 0.9843 - val_loss: 0.0734 - val_accuracy: 0.9800
Epoch 6/20
1183/1183 [==============================] - 37s 31ms/step - loss: 0.0454 - accuracy: 0.9863 -

In [ ]:
class_0 = model0.load_model("final_models/1anomaly-classifier0")
class_1 = model1.load_model("final_models/1anomaly-classifier1")
class_2 = model2.load_model("final_models/1anomaly-classifier2")
class_3 = model3.load_model("final_models/1anomaly-classifier3")
class_4 = model4.load_model("final_models/1anomaly-classifier4")
class_5 = model5.load_model("final_models/1anomaly-classifier5")
class_6 = model6.load_model("final_models/1anomaly-classifier6")
class_7 = model7.load_model("final_models/1anomaly-classifier7")
class_8 = model8.load_model("final_models/1anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_61', 'dense_63', 'dense_65', 'dense_67', 'dense_69', 'dense_71', 'dense_73', 'dense_75', 'dense_19']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9503,)
Temperature Initial value: 1.0
Temperature Final value: 2.06091570854187
{0: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9504,)
Temperature Initial value: 1.0
Temperature Final value: 2.3180222511291504
{0: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9459,)
Temperature Initial value: 1.0
Temperature Final value: 2.2263615131378174
{0: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9502,)
Temperature Initial value: 1.0
Temperature Final value: 2.51077938079834
{0: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9574,)
Temperature Initial value: 1.0
Temperature Final value: 2.166799545288086
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9501,)
Temperature Initial value: 1.0
Temperature Final value: 2.330876111984253
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4,

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0609157>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3180223>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2263615>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.5107794>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1667995>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.330876>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9582385>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1498156>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.477948>]

In [ ]:
temp_val = [2.0609157, 2.3180223, 2.2263615, 2.5107794, 2.1667995, 2.330876, 1.9582385, 2.1498156, 2.477948]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 2:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10678it [12:39, 14.06it/s]

[9347.98394420743, 9278.456224650145, 9234.90728765726, 9345.79588869214, 9441.71300148964, 9293.35782277584, 9240.628234177828, 9357.519878834486, 9342.639295220375]
[9503, 9504, 9459, 9502, 9574, 9501, 9379, 9518, 9484]
[496.24585637980846, 682.8849439422588, 687.3766117310041, 461.4241289756502, 409.26674899533475, 629.3891915531387, 442.9177857183027, 493.7653927540591, 417.6592559437346]
Entropy: 0.05527045100628685
Each Classifier Average:  [0.9836876717044544, 0.9762685421559496, 0.9763090482775411, 0.9835609228259462, 0.9861826824200585, 0.9781452292154342, 0.98524663974601, 0.9831393022519948, 0.9850948223555858]
Threshold: 0.981959428994775


In [ ]:
ref_vect_in_1an = []
ref_vect_in_1an.append(classifier_avg_t)

threshold_in_1an = []
threshold_in_1an.append(treshold_t)

entropy_in_1an = []
entropy_in_1an.append(entropy_t)

print(entropy_in_1an)
print(ref_vect_in_1an)
print(threshold_in_1an)

[0.05527045100628685]
[[0.9836876717044544, 0.9762685421559496, 0.9763090482775411, 0.9835609228259462, 0.9861826824200585, 0.9781452292154342, 0.98524663974601, 0.9831393022519948, 0.9850948223555858]]
[0.981959428994775]


In [ ]:
entropy_in_1an = [0.05527045100628685]
ref_vect_in_1an = [[0.9836876717044544, 0.9762685421559496, 0.9763090482775411, 0.9835609228259462, 0.9861826824200585, 0.9781452292154342, 0.98524663974601, 0.9831393022519948, 0.9850948223555858]]
threshold_in_1an = [0.981959428994775]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 2:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10678it [01:34, 112.98it/s]

[941.3688916862011, 915.7900085449219, 970.7751380354166, 1043.6230748593807, 865.9130089879036, 893.5144730508327, 1028.0956514179707, 886.7211111187935, 1091.8376369774342]
[1175, 1174, 1219, 1176, 1104, 1177, 1299, 1160, 1194]
[592.6858783224889, 687.8466444238438, 658.8614348977835, 354.9468186097744, 621.0151225871814, 727.5807700690093, 741.4647662199568, 729.2438900739653, 280.5161680721851]
Entropy: 0.5053005462416496
Each Classifier Average:  [0.8011650142010223, 0.7800596324914155, 0.7963700886262647, 0.8874345874654598, 0.7843414936484634, 0.7591456865342674, 0.7914516177197619, 0.7644147509644772, 0.9144368818906484]
Threshold: 0.8087577503935313


In [ ]:
ref_vect_out_1an = []
ref_vect_out_1an.append(classifier_avg_t)

threshold_out_1an = []
threshold_out_1an.append(treshold_t)

entropy_out_1an = []
entropy_out_1an.append(entropy_t)

print(entropy_out_1an)
print(ref_vect_out_1an)
print(threshold_out_1an)

[0.5053005462416496]
[[0.8011650142010223, 0.7800596324914155, 0.7963700886262647, 0.8874345874654598, 0.7843414936484634, 0.7591456865342674, 0.7914516177197619, 0.7644147509644772, 0.9144368818906484]]
[0.8087577503935313]


In [ ]:
entropy_out_1an = [0.5053005462416496]
ref_vect_out_1an = [[0.8011650142010223, 0.7800596324914155, 0.7963700886262647, 0.8874345874654598, 0.7843414936484634, 0.7591456865342674, 0.7914516177197619, 0.7644147509644772, 0.9144368818906484]]
threshold_out_1an = [0.8087577503935313]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
set(train_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_1an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 1:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 1)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(2, 1)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(3, 1)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(4, 1)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(5, 1)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 1)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 1)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 1)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 1)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_1an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:53,  7.98it/s]


In [ ]:
pickle_out = open("sm_all_wt_1an.pickle","wb")
pickle.dump(max_sm_all_wt_1an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_1an.pickle","rb")
max_sm_all_wt_1an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_1an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 9, 7, 7] 1 7


In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_1an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_1an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_1an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_1an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_1an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)

  if sim_in >= sim_ood:
    y_ = ind    
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_1an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9159
Accuracy of determining ID data:  0.9054709531866892
Accuracy of determining OOD data:  0.9973568281938326


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9949769300148334

In [ ]:
roc_auc_score(y_true, y_pred)

0.951413890690261

In [ ]:
roc_auc_score(y_true, sim_score_)

0.9425150135040785

# 2 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(2, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='2anomaly:classifier0')
model1 = Model(name='2anomaly:classifier1')
model2 = Model(name='2anomaly:classifier2')
model3 = Model(name='2anomaly:classifier3')
model4 = Model(name='2anomaly:classifier4')
model5 = Model(name='2anomaly:classifier5')
model6 = Model(name='2anomaly:classifier6')
model7 = Model(name='2anomaly:classifier7')
model8 = Model(name='2anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/2anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/2anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(3, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(3, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/2anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(4, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(4, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/2anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(5, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(5, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/2anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/2anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/2anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/2anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/2anomaly-classifier8")
print('Model 8 saved')

{1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38468,)
{1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9651,)
Epoch 1/20
1203/1203 [==============================] - 39s 32ms/step - loss: 5.3457 - accuracy: 0.8408 - val_loss: 0.2821 - val_accuracy: 0.9575
Epoch 2/20
1203/1203 [==============================] - 38s 31ms/step - loss: 0.1390 - accuracy: 0.9737 - val_loss: 0.1374 - val_accuracy: 0.9702
Epoch 3/20
1203/1203 [==============================] - 38s 31ms/step - loss: 0.0704 - accuracy: 0.9810 - val_loss: 0.0665 - val_accuracy: 0.9810
Epoch 4/20
1203/1203 [==============================] - 38s 31ms/step - loss: 0.0636 - accuracy: 0.9808 - val_loss: 0.1759 - val_accuracy: 0.9483
Epoch 5/20
1203/1203 [==============================] - 38s 31ms/step - loss: 0.0542 - accuracy: 0.9824 - val_loss: 0.1057 - val_accuracy: 0.9712
Epoch 6/20
1203/1203 [==============================] - 38s 31ms/step - loss: 0.0493 - accuracy: 0.9847 -

In [ ]:
class_0 = model0.load_model("final_models/2anomaly-classifier0")
class_1 = model1.load_model("final_models/2anomaly-classifier1")
class_2 = model2.load_model("final_models/2anomaly-classifier2")
class_3 = model3.load_model("final_models/2anomaly-classifier3")
class_4 = model4.load_model("final_models/2anomaly-classifier4")
class_5 = model5.load_model("final_models/2anomaly-classifier5")
class_6 = model6.load_model("final_models/2anomaly-classifier6")
class_7 = model7.load_model("final_models/2anomaly-classifier7")
class_8 = model8.load_model("final_models/2anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9651,)
Temperature Initial value: 1.0
Temperature Final value: 2.2466559410095215
{0: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9504,)
Temperature Initial value: 1.0
Temperature Final value: 2.210195302963257
{0: 0, 1: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9607,)
Temperature Initial value: 1.0
Temperature Final value: 2.264735460281372
{0: 0, 1: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9650,)
Temperature Initial value: 1.0
Temperature Final value: 2.2229294776916504
{0: 0, 1: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9722,)
Temperature Initial value: 1.0
Temperature Final value: 2.145726442337036
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9649,)
Temperature Initial value: 1.0
Temperature Final value: 2.14382266998291
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.246656>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2101953>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2647355>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2229295>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1457264>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1438227>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1131978>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0234013>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4829793>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10826it [12:16, 14.71it/s]

[9405.049283653498, 9348.631170630455, 9351.427759766579, 9434.45449385047, 9538.491575554013, 9528.481483578682, 9328.142667412758, 9555.529257655144, 9467.76197116077]
[9651, 9504, 9607, 9650, 9722, 9649, 9527, 9666, 9632]
[759.5902696749478, 465.77641399661775, 758.9109294853397, 620.9661511954807, 577.9073483017497, 376.98345040381264, 591.5475185285524, 338.60580607712694, 502.7499658321673]
Entropy: 0.05765442328513453
Each Classifier Average:  [0.9745155200138326, 0.9836522696370428, 0.9733972894521264, 0.9776636781192196, 0.9811244163293574, 0.9875097402403028, 0.9791269725425378, 0.9885712039783927, 0.9829487096304785]
Threshold: 0.9809455333270323


In [ ]:
ref_vect_in_2an = []
ref_vect_in_2an.append(classifier_avg_t)

threshold_in_2an = []
threshold_in_2an.append(treshold_t)

entropy_in_2an = []
entropy_in_2an.append(entropy_t)

print(entropy_in_2an)
print(ref_vect_in_2an)
print(threshold_in_2an)

[0.05765442328513453]
[[0.9745155200138326, 0.9836522696370428, 0.9733972894521264, 0.9776636781192196, 0.9811244163293574, 0.9875097402403028, 0.9791269725425378, 0.9885712039783927, 0.9829487096304785]]
[0.9809455333270323]


In [ ]:
entropy_in_2an = [0.05765442328513453]
ref_vect_in_2an = [[0.9745155200138326, 0.9836522696370428, 0.9733972894521264, 0.9776636781192196, 0.9811244163293574, 0.9875097402403028, 0.9791269725425378, 0.9885712039783927, 0.9829487096304785]]
threshold_in_2an = [0.9809455333270323]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 3:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10826it [01:33, 116.08it/s]

[989.6919656097889, 967.0002437233925, 985.5703043341637, 958.5422583520412, 912.68167501688, 939.2893269062042, 1002.4835924506187, 878.8140273094177, 946.6589447259903]
[1175, 1322, 1219, 1176, 1104, 1177, 1299, 1160, 1194]
[503.93807428907076, 1004.7887137681246, 614.641632284809, 598.5627212467662, 497.61645826513995, 607.1953896166997, 728.5454240590625, 741.3068669619388, 660.052928874884]
Entropy: 0.5468306124703104
Each Classifier Average:  [0.8422910345615224, 0.7314676578845631, 0.808507222587501, 0.8150869543809874, 0.8267044157761595, 0.7980368113051862, 0.7717348671675279, 0.7575982994046705, 0.7928466873752013]
Threshold: 0.7938082167159244


In [ ]:
ref_vect_out_2an = []
ref_vect_out_2an.append(classifier_avg_t)

threshold_out_2an = []
threshold_out_2an.append(treshold_t)

entropy_out_2an = []
entropy_out_2an.append(entropy_t)

print(entropy_out_2an)
print(ref_vect_out_2an)
print(threshold_out_2an)

[0.5468306124703104]
[[0.8422910345615224, 0.7314676578845631, 0.808507222587501, 0.8150869543809874, 0.8267044157761595, 0.7980368113051862, 0.7717348671675279, 0.7575982994046705, 0.7928466873752013]]
[0.7938082167159244]


In [ ]:
entropy_out_2an = [0.5468306124703104]
ref_vect_out_2an = [[0.8422910345615224, 0.7314676578845631, 0.808507222587501, 0.8150869543809874, 0.8267044157761595, 0.7980368113051862, 0.7717348671675279, 0.7575982994046705, 0.7928466873752013]]
threshold_out_2an = [0.7938082167159244]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_2an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 2:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 0)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 0)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(3, 0)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(4, 0)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(5, 0)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 0)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 0)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 0)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 0)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_2an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:03,  8.31it/s]


In [ ]:
pickle_out = open("sm_all_wt_2an.pickle","wb")
pickle.dump(max_sm_all_wt_2an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_2an.pickle","rb")
max_sm_all_wt_2an = pickle.load(pickle_in)

In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_2an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_2an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_2an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_2an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_2an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)

  if sim_in >= sim_ood:
    y_ = ind    
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_2an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9091
Accuracy of determining ID data:  0.90421498661909
Accuracy of determining OOD data:  0.9515503875968992


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9580710960244521

In [ ]:
roc_auc_score(y_true, y_pred)

0.9278826871079946

In [ ]:
roc_auc_score(y_true, sim_score_)

0.9596073506835673

# 3 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(3, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='3anomaly:classifier0')
model1 = Model(name='3anomaly:classifier1')
model2 = Model(name='3anomaly:classifier2')
model3 = Model(name='3anomaly:classifier3')
model4 = Model(name='3anomaly:classifier4')
model5 = Model(name='3anomaly:classifier5')
model6 = Model(name='3anomaly:classifier6')
model7 = Model(name='3anomaly:classifier7')
model8 = Model(name='3anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/3anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/3anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/3anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(4, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(4, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/3anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(5, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(5, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/3anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/3anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/3anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/3anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/3anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38340,)
{1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9606,)
Epoch 1/20
1199/1199 [==============================] - 45s 31ms/step - loss: 3.5449 - accuracy: 0.8636 - val_loss: 0.1443 - val_accuracy: 0.9666
Epoch 2/20
1199/1199 [==============================] - 38s 32ms/step - loss: 0.1130 - accuracy: 0.9732 - val_loss: 0.0815 - val_accuracy: 0.9738
Epoch 3/20
1199/1199 [==============================] - 39s 33ms/step - loss: 0.0700 - accuracy: 0.9803 - val_loss: 0.0888 - val_accuracy: 0.9740
Epoch 4/20
1199/1199 [==============================] - 39s 32ms/step - loss: 0.0593 - accuracy: 0.9821 - val_loss: 0.2302 - val_accuracy: 0.9469
Epoch 5/20
1199/1199 [==============================] - 39s 32ms/step - loss: 0.0583 - accuracy: 0.9825 - val_loss: 0.0800 - val_accuracy: 0.9766
Epoch 6/20
1199/1199 [==============================] - 39s 32ms/step - loss: 0.0602 - accuracy: 0.9819 -

In [ ]:
class_0 = model0.load_model("final_models/3anomaly-classifier0")
class_1 = model1.load_model("final_models/3anomaly-classifier1")
class_2 = model2.load_model("final_models/3anomaly-classifier2")
class_3 = model3.load_model("final_models/3anomaly-classifier3")
class_4 = model4.load_model("final_models/3anomaly-classifier4")
class_5 = model5.load_model("final_models/3anomaly-classifier5")
class_6 = model6.load_model("final_models/3anomaly-classifier6")
class_7 = model7.load_model("final_models/3anomaly-classifier7")
class_8 = model8.load_model("final_models/3anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_19', 'dense_21', 'dense_23', 'dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9606,)
Temperature Initial value: 1.0
Temperature Final value: 1.931548833847046
{0: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9459,)
Temperature Initial value: 1.0
Temperature Final value: 2.004016637802124
{0: 0, 1: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9607,)
Temperature Initial value: 1.0
Temperature Final value: 1.7066195011138916
{0: 0, 1: 1, 2: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9605,)
Temperature Initial value: 1.0
Temperature Final value: 2.3148810863494873
{0: 0, 1: 1, 2: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9677,)
Temperature Initial value: 1.0
Temperature Final value: 2.081744909286499
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9604,)
Temperature Initial value: 1.0
Temperature Final value: 1.9893512725830078
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3,

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9315488>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0040166>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.7066195>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.314881>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.081745>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9893513>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4744115>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3960419>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1575763>]

In [ ]:
temp_val = [1.9315488, 2.0040166, 1.7066195, 2.314881, 2.081745, 1.9893513, 2.4744115, 2.3960419, 2.1575763]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10781it [12:24, 14.48it/s]

[9447.016822248697, 9301.52622115612, 9468.218563318253, 9457.007133245468, 9546.94389435649, 9412.477506011724, 9310.00702676177, 9456.177287399769, 9463.62655544281]
[9606, 9459, 9607, 9605, 9677, 9604, 9482, 9621, 9587]
[508.6683546499203, 473.8475824471871, 462.0240203240533, 439.62953608612577, 415.68070369158715, 608.3904361484526, 474.91548737375604, 524.5318875828478, 400.0190230457493]
Entropy: 0.04994946632199426
Each Classifier Average:  [0.9834495963198727, 0.983351963331866, 0.985554133789763, 0.9845921013269618, 0.9865602866959273, 0.9800580493556564, 0.9818611080744326, 0.9828684427190281, 0.9871311729887149]
Threshold: 0.9839363171780248


In [ ]:
ref_vect_in_3an = []
ref_vect_in_3an.append(classifier_avg_t)

threshold_in_3an = []
threshold_in_3an.append(treshold_t)

entropy_in_3an = []
entropy_in_3an.append(entropy_t)

print(entropy_in_3an)
print(ref_vect_in_3an)
print(threshold_in_3an)

[0.04994946632199426]
[[0.9834495963198727, 0.983351963331866, 0.985554133789763, 0.9845921013269618, 0.9865602866959273, 0.9800580493556564, 0.9818611080744326, 0.9828684427190281, 0.9871311729887149]]
[0.9839363171780248]


In [ ]:
entropy_in_3an = [0.04994946632199426]
ref_vect_in_3an = [[0.9834495963198727, 0.983351963331866, 0.985554133789763, 0.9845921013269618, 0.9865602866959273, 0.9800580493556564, 0.9818611080744326, 0.9828684427190281, 0.9871311729887149]]
threshold_in_3an = [0.9839363171780248]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 4:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10781it [01:33, 115.22it/s]

[932.4524344801903, 802.844724625349, 986.0760701298714, 962.5460428744555, 872.8528325855732, 1016.6342995464802, 1052.8210000097752, 932.3528594970703, 1100.4509341716766]
[1175, 1322, 1174, 1176, 1104, 1177, 1299, 1160, 1194]
[636.5551197232853, 1377.858542952701, 483.8661005220247, 570.7016475885794, 598.2657914990777, 442.29709160014136, 627.5183741055953, 568.9808160095199, 261.42190548987924]
Entropy: 0.5101844694644414
Each Classifier Average:  [0.7935765399831407, 0.6072955556923971, 0.8399285094802993, 0.8184915330565098, 0.7906275657478018, 0.8637504669044012, 0.8104857582831217, 0.8037524650836813, 0.921650698636245]
Threshold: 0.8055065658741776


In [ ]:
ref_vect_out_3an = []
ref_vect_out_3an.append(classifier_avg_t)

threshold_out_3an = []
threshold_out_3an.append(treshold_t)

entropy_out_3an = []
entropy_out_3an.append(entropy_t)

print(entropy_out_3an)
print(ref_vect_out_3an)
print(threshold_out_3an)

[0.5101844694644414]
[[0.7935765399831407, 0.6072955556923971, 0.8399285094802993, 0.8184915330565098, 0.7906275657478018, 0.8637504669044012, 0.8104857582831217, 0.8037524650836813, 0.921650698636245]]
[0.8055065658741776]


In [ ]:
entropy_out_3an = [0.5101844694644414]
ref_vect_out_3an = [[0.7935765399831407, 0.6072955556923971, 0.8399285094802993, 0.8184915330565098, 0.7906275657478018, 0.8637504669044012, 0.8104857582831217, 0.8037524650836813, 0.921650698636245]]
threshold_out_3an = [0.8055065658741776]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_3an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 3:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 3)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 3)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 3)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(4, 3)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(5, 3)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 3)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 3)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 3)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 3)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_3an.append([ref_value, lbl, pred_value, label, e_])


Streaming output truncated to the last 5000 lines.
5001it [11:44,  6.52it/s]
5002it [11:44,  6.56it/s]
5003it [11:44,  6.51it/s]
5004it [11:44,  6.41it/s]
5005it [11:45,  6.42it/s]
5006it [11:45,  6.39it/s]
5007it [11:45,  6.49it/s]
5008it [11:45,  6.47it/s]
5009it [11:45,  6.46it/s]
5010it [11:45,  6.35it/s]
5011it [11:46,  6.52it/s]
5012it [11:46,  6.65it/s]
5013it [11:46,  6.59it/s]
5014it [11:46,  6.59it/s]
5015it [11:46,  6.54it/s]
5016it [11:46,  6.53it/s]
5017it [11:46,  6.39it/s]
5018it [11:47,  6.53it/s]
5019it [11:47,  6.46it/s]
5020it [11:47,  6.42it/s]
5021it [11:47,  6.50it/s]
5022it [11:47,  6.53it/s]
5023it [11:47,  6.51it/s]
5024it [11:48,  6.62it/s]
5025it [11:48,  6.68it/s]
5026it [11:48,  6.76it/s]
5027it [11:48,  6.81it/s]
5028it [11:48,  6.81it/s]
5029it [11:48,  6.93it/s]
5030it [11:48,  6.88it/s]
5031it [11:49,  6.86it/s]
5032it [11:49,  6.87it/s]
5033it [11:49,  6.90it/s]
5034it [11:49,  6.90it/s]
5035it [11:49,  6.78it/s]
5036it [11:49,  6.82it/s]
5037it [11:49

In [ ]:
pickle_out = open("sm_all_wt_3an.pickle","wb")
pickle.dump(max_sm_all_wt_3an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_3an.pickle","rb")
max_sm_all_wt_3an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_3an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 9, 7, 7] 1 7


In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_3an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_3an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_3an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_3an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_3an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
#  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in >= sim_ood:
    y_ = ind    
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_3an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9252
Accuracy of determining ID data:  0.9407119021134595
Accuracy of determining OOD data:  0.7871287128712872


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9341258163636164

In [ ]:
roc_auc_score(y_true, y_pred)

0.8945252701020937

In [ ]:
roc_auc_score(y_true, sim_score_)

0.8732751462020506

# 4 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(4, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='4anomaly:classifier0')
model1 = Model(name='4anomaly:classifier1')
model2 = Model(name='4anomaly:classifier2')
model3 = Model(name='4anomaly:classifier3')
model4 = Model(name='4anomaly:classifier4')
model5 = Model(name='4anomaly:classifier5')
model6 = Model(name='4anomaly:classifier6')
model7 = Model(name='4anomaly:classifier7')
model8 = Model(name='4anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/4anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/4anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/4anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/4anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(5, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(5, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/4anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/4anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/4anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/4anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/4anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38586,)
{1: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9649,)
Epoch 1/20
1206/1206 [==============================] - 39s 31ms/step - loss: 6.1452 - accuracy: 0.8297 - val_loss: 0.2764 - val_accuracy: 0.9518
Epoch 2/20
1206/1206 [==============================] - 38s 32ms/step - loss: 0.1667 - accuracy: 0.9666 - val_loss: 0.1649 - val_accuracy: 0.9626
Epoch 3/20
1206/1206 [==============================] - 39s 32ms/step - loss: 0.0720 - accuracy: 0.9792 - val_loss: 0.1293 - val_accuracy: 0.9620
Epoch 4/20
1206/1206 [==============================] - 38s 32ms/step - loss: 0.0554 - accuracy: 0.9839 - val_loss: 0.1428 - val_accuracy: 0.9645
Epoch 5/20
1206/1206 [==============================] - 39s 32ms/step - loss: 0.0510 - accuracy: 0.9848 - val_loss: 0.1995 - val_accuracy: 0.9412
Epoch 6/20
1206/1206 [==============================] - 39s 32ms/step - loss: 0.0483 - accuracy: 0.9848 -

In [ ]:
class_0 = model0.load_model("final_models/4anomaly-classifier0")
class_1 = model1.load_model("final_models/4anomaly-classifier1")
class_2 = model2.load_model("final_models/4anomaly-classifier2")
class_3 = model3.load_model("final_models/4anomaly-classifier3")
class_4 = model4.load_model("final_models/4anomaly-classifier4")
class_5 = model5.load_model("final_models/4anomaly-classifier5")
class_6 = model6.load_model("final_models/4anomaly-classifier6")
class_7 = model7.load_model("final_models/4anomaly-classifier7")
class_8 = model8.load_model("final_models/4anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
set(new_val_labels)

{0, 1, 2, 3, 5, 6, 7, 8, 9}

In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9649,)
Temperature Initial value: 1.0
Temperature Final value: 2.173856735229492
{0: 0, 2: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9502,)
Temperature Initial value: 1.0
Temperature Final value: 2.337050676345825
{0: 0, 1: 1, 3: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9650,)
Temperature Initial value: 1.0
Temperature Final value: 2.1365163326263428
{0: 0, 1: 1, 2: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9605,)
Temperature Initial value: 1.0
Temperature Final value: 2.4419920444488525
{0: 0, 1: 1, 2: 2, 3: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9720,)
Temperature Initial value: 1.0
Temperature Final value: 2.2172510623931885
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9647,)
Temperature Initial value: 1.0
Temperature Final value: 2.4548637866973877
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1738567>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3370507>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1365163>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.441992>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.217251>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4548638>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1809883>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.343067>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4618614>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10824it [13:03, 13.81it/s]

[9500.433717489243, 9309.620541930199, 9436.483246862888, 9380.95329490304, 9589.341369599104, 9441.919467359781, 9414.486594974995, 9492.813774943352, 9409.799656361341]
[9649, 9502, 9650, 9605, 9720, 9647, 9525, 9664, 9630]
[464.3935592266183, 576.7322754964854, 660.4414740861491, 696.0975343614757, 436.36395912858495, 619.2559443452235, 337.89602810794224, 526.2041780826671, 642.5170021022818]
Entropy: 0.057274044994271946
Each Classifier Average:  [0.9846029347589639, 0.9797537930888444, 0.9778739115920092, 0.9766739505364955, 0.9865577540739819, 0.9787415224795046, 0.988397542779527, 0.9822861935992707, 0.9771339207021124]
Threshold: 0.9813357248456344


In [ ]:
ref_vect_in_4an = []
ref_vect_in_4an.append(classifier_avg_t)

threshold_in_4an = []
threshold_in_4an.append(treshold_t)

entropy_in_4an = []
entropy_in_4an.append(entropy_t)

print(entropy_in_4an)
print(ref_vect_in_4an)
print(threshold_in_4an)

[0.057274044994271946]
[[0.9846029347589639, 0.9797537930888444, 0.9778739115920092, 0.9766739505364955, 0.9865577540739819, 0.9787415224795046, 0.988397542779527, 0.9822861935992707, 0.9771339207021124]]
[0.9813357248456344]


In [ ]:
entropy_in_4an = [0.057274044994271946]
ref_vect_in_4an = [[0.9846029347589639, 0.9797537930888444, 0.9778739115920092, 0.9766739505364955, 0.9865577540739819, 0.9787415224795046, 0.988397542779527, 0.9822861935992707, 0.9771339207021124]]
threshold_in_4an = [0.9813357248456344]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 5:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10824it [01:38, 109.75it/s]

[1013.7186418324709, 1118.7598364055157, 935.0846498310566, 867.7272661179304, 833.4998598992825, 964.6141183376312, 1123.734256207943, 937.2166780531406, 911.3080673217773]
[1175, 1322, 1174, 1219, 1104, 1177, 1299, 1160, 1194]
[415.5636234981016, 633.170321919024, 634.4051616337965, 945.1361446573865, 701.7003532266244, 586.3620913837574, 459.16280535294715, 580.1264853518418, 756.3519456838559]
Entropy: 0.5299068746094104
Each Classifier Average:  [0.8627392696446561, 0.8462631137711919, 0.7964945909974929, 0.711835329054906, 0.7549817571551471, 0.8195532016462457, 0.8650764097058837, 0.8079454121147764, 0.7632395873716729]
Threshold: 0.8031254079402191


In [ ]:
ref_vect_out_4an = []
ref_vect_out_4an.append(classifier_avg_t)

threshold_out_4an = []
threshold_out_4an.append(treshold_t)

entropy_out_4an = []
entropy_out_4an.append(entropy_t)

print(entropy_out_4an)
print(ref_vect_out_4an)
print(threshold_out_4an)

[0.5299068746094104]
[[0.8627392696446561, 0.8462631137711919, 0.7964945909974929, 0.711835329054906, 0.7549817571551471, 0.8195532016462457, 0.8650764097058837, 0.8079454121147764, 0.7632395873716729]]
[0.8031254079402191]


In [ ]:
entropy_out_4an = [0.5299068746094104]
ref_vect_out_4an = [[0.8627392696446561, 0.8462631137711919, 0.7964945909974929, 0.711835329054906, 0.7549817571551471, 0.8195532016462457, 0.8650764097058837, 0.8079454121147764, 0.7632395873716729]]
threshold_out_4an = [0.8031254079402191]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_4an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 4:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 4)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 4)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 4)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 4)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(5, 4)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 4)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 4)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 4)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 4)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_4an.append([ref_value, lbl, pred_value, label, e_])


10000it [21:18,  7.82it/s]


In [ ]:
pickle_out = open("sm_all_wt_4an.pickle","wb")
pickle.dump(max_sm_all_wt_4an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_4an.pickle","rb")
max_sm_all_wt_4an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_4an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 2, 7, 7] 1 7


In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_4an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_4an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_4an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_4an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_4an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in >= sim_ood:
    y_ = ind    
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_4an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.8812
Accuracy of determining ID data:  0.9045242847638058
Accuracy of determining OOD data:  0.6670061099796334


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.8662938888008098

In [ ]:
roc_auc_score(y_true, y_pred)

0.7655492364445131

In [ ]:
roc_auc_score(y_true, sim_score_)

0.8635207521142373

# 5 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(5, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='5anomaly:classifier0')
model1 = Model(name='5anomaly:classifier1')
model2 = Model(name='5anomaly:classifier2')
model3 = Model(name='5anomaly:classifier3')
model4 = Model(name='5anomaly:classifier4')
model5 = Model(name='5anomaly:classifier5')
model6 = Model(name='5anomaly:classifier6')
model7 = Model(name='5anomaly:classifier7')
model8 = Model(name='5anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/5anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/5anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/5anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/5anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(4, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/5anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(6, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(6, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/5anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/5anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/5anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/5anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38935,)
{1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9721,)
Epoch 1/20
1217/1217 [==============================] - 38s 31ms/step - loss: 4.4210 - accuracy: 0.8301 - val_loss: 0.1378 - val_accuracy: 0.9621
Epoch 2/20
1217/1217 [==============================] - 37s 31ms/step - loss: 0.1204 - accuracy: 0.9665 - val_loss: 0.1882 - val_accuracy: 0.9388
Epoch 3/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.0802 - accuracy: 0.9769 - val_loss: 0.1277 - val_accuracy: 0.9697
Epoch 4/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.0655 - accuracy: 0.9799 - val_loss: 0.1438 - val_accuracy: 0.9671
Epoch 5/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.0599 - accuracy: 0.9829 - val_loss: 0.1481 - val_accuracy: 0.9647
Epoch 6/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.0593 - accuracy: 0.9821 -

In [ ]:
#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/5anomaly-classifier8")
print('Model 8 saved')

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38928,)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9702,)
Epoch 1/20
1217/1217 [==============================] - 45s 31ms/step - loss: 5.0705 - accuracy: 0.8361 - val_loss: 0.2271 - val_accuracy: 0.9683
Epoch 2/20
1217/1217 [==============================] - 37s 31ms/step - loss: 0.1265 - accuracy: 0.9743 - val_loss: 0.1462 - val_accuracy: 0.9758
Epoch 3/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.1039 - accuracy: 0.9764 - val_loss: 0.1703 - val_accuracy: 0.9723
Epoch 4/20
1217/1217 [==============================] - 38s 31ms/step - loss: 0.0832 - accuracy: 0.9792 - val_loss: 0.1113 - val_accuracy: 0.9743
Epoch 5/20
1217/1217 [==============================] - 39s 32ms/step - loss: 0.0536 - accuracy: 0.9845 - val_loss: 0.0985 - val_accuracy: 0.9741
Epoch 6/20
1217/1217 [==============================] - 39s 32ms/step - loss: 0.0568 - accuracy: 0.9835 -

In [ ]:
class_0 = model0.load_model("final_models/5anomaly-classifier0")
class_1 = model1.load_model("final_models/5anomaly-classifier1")
class_2 = model2.load_model("final_models/5anomaly-classifier2")
class_3 = model3.load_model("final_models/5anomaly-classifier3")
class_4 = model4.load_model("final_models/5anomaly-classifier4")
class_5 = model5.load_model("final_models/5anomaly-classifier5")
class_6 = model6.load_model("final_models/5anomaly-classifier6")
class_7 = model7.load_model("final_models/5anomaly-classifier7")
class_8 = model8.load_model("final_models/5anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_93', 'dense_95', 'dense_97', 'dense_99', 'dense_101', 'dense_103', 'dense_105', 'dense_107', 'dense_19']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
set(new_val_labels)

{0, 1, 2, 3, 4, 6, 7, 8, 9}

In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9721,)
Temperature Initial value: 1.0
Temperature Final value: 2.056544065475464
{0: 0, 2: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9574,)
Temperature Initial value: 1.0
Temperature Final value: 2.1163601875305176
{0: 0, 1: 1, 3: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9722,)
Temperature Initial value: 1.0
Temperature Final value: 2.3622562885284424
{0: 0, 1: 1, 2: 2, 4: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9677,)
Temperature Initial value: 1.0
Temperature Final value: 2.390667676925659
{0: 0, 1: 1, 2: 2, 3: 3, 6: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9720,)
Temperature Initial value: 1.0
Temperature Final value: 2.2536275386810303
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9719,)
Temperature Initial value: 1.0
Temperature Final value: 2.30367112159729
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.056544>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1163602>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3622563>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3906677>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2536275>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3036711>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.5086586>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2106051>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.266882>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10896it [12:50, 14.14it/s]

[9603.444884419441, 9458.451500624418, 9540.7890779078, 9502.559653133154, 9601.45196017623, 9567.101613849401, 9315.152011096478, 9607.079118192196, 9541.850670456886]
[9721, 9574, 9722, 9677, 9720, 9719, 9597, 9736, 9702]
[372.91374230632096, 363.7050214233947, 551.1446032763299, 553.0551536597962, 369.17797936960636, 473.1718330646984, 815.7394185809904, 390.944743651228, 520.7509795909762]
Entropy: 0.050631949011271875
Each Classifier Average:  [0.9879070964324083, 0.9879310111368726, 0.981360736258774, 0.9819737163514678, 0.987803699606608, 0.9843709860941868, 0.970631656881992, 0.9867583317781631, 0.9834931633123981]
Threshold: 0.9835811553169856


In [ ]:
ref_vect_in_5an = []
ref_vect_in_5an.append(classifier_avg_t)

threshold_in_5an = []
threshold_in_5an.append(treshold_t)

entropy_in_5an = []
entropy_in_5an.append(entropy_t)

print(entropy_in_5an)
print(ref_vect_in_5an)
print(threshold_in_5an)

[0.050631949011271875]
[[0.9879070964324083, 0.9879310111368726, 0.981360736258774, 0.9819737163514678, 0.987803699606608, 0.9843709860941868, 0.970631656881992, 0.9867583317781631, 0.9834931633123981]]
[0.9835811553169856]


In [ ]:
entropy_in_5an = [0.050631949011271875]
ref_vect_in_5an = [[0.9879070964324083, 0.9879310111368726, 0.981360736258774, 0.9819737163514678, 0.987803699606608, 0.9843709860941868, 0.970631656881992, 0.9867583317781631, 0.9834931633123981]]
threshold_in_5an = [0.9835811553169856]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 6:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10896it [01:37, 111.86it/s]

[924.3241517841816, 731.6788532584906, 946.221238464117, 1028.1971726715565, 1016.7988847345114, 983.6910042464733, 949.471155077219, 902.6606788635254, 936.6449221968651]
[1175, 1322, 1174, 1219, 1176, 1177, 1299, 1160, 1194]
[669.8063464202714, 1523.4804593534209, 594.4251946387958, 508.0769529964525, 438.64450425197083, 511.1045718697294, 898.2843052904627, 683.4913887701114, 597.5700110852486]
Entropy: 0.5815590091425396
Each Classifier Average:  [0.7866588525822822, 0.553463580377073, 0.8059806119796568, 0.8434759414861005, 0.8646249019851288, 0.8357612610420334, 0.7309246767338099, 0.7781557576409701, 0.7844597338332203]
Threshold: 0.7759450352955861


In [ ]:
ref_vect_out_5an = []
ref_vect_out_5an.append(classifier_avg_t)

threshold_out_5an = []
threshold_out_5an.append(treshold_t)

entropy_out_5an = []
entropy_out_5an.append(entropy_t)

print(entropy_out_5an)
print(ref_vect_out_5an)
print(threshold_out_5an)

[0.5815590091425396]
[[0.7866588525822822, 0.553463580377073, 0.8059806119796568, 0.8434759414861005, 0.8646249019851288, 0.8357612610420334, 0.7309246767338099, 0.7781557576409701, 0.7844597338332203]]
[0.7759450352955861]


In [ ]:
entropy_out_5an = [0.5815590091425396]
ref_vect_out_5an = [[0.7866588525822822, 0.553463580377073, 0.8059806119796568, 0.8434759414861005, 0.8646249019851288, 0.8357612610420334, 0.7309246767338099, 0.7781557576409701, 0.7844597338332203]]
threshold_out_5an = [0.7759450352955861]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_5an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 5:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 5)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 5)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 5)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 5)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(4, 5)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(6, 5)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 5)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 5)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 5)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_5an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:53,  7.98it/s]


In [ ]:
pickle_out = open("sm_all_wt_5an.pickle","wb")
pickle.dump(max_sm_all_wt_5an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_5an.pickle","rb")
max_sm_all_wt_5an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_5an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 9, 7, 7] 1 7


In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_5an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_5an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_5an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_5an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_5an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in >= sim_ood:
    y_ = ind    
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_5an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9271
Accuracy of determining ID data:  0.9236934563021519
Accuracy of determining OOD data:  0.9618834080717489


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9603154030064734

In [ ]:
roc_auc_score(y_true, y_pred)

0.9427884321869504

In [ ]:
roc_auc_score(y_true, sim_score_)

0.9628085298293915

# 6 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(6, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='6anomaly:classifier0')
model1 = Model(name='6anomaly:classifier1')
model2 = Model(name='6anomaly:classifier2')
model3 = Model(name='6anomaly:classifier3')
model4 = Model(name='6anomaly:classifier4')
model5 = Model(name='6anomaly:classifier5')
model6 = Model(name='6anomaly:classifier6')
model7 = Model(name='6anomaly:classifier7')
model8 = Model(name='6anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/6anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/6anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/6anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/6anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(4, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/6anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(5, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(5, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/6anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(7, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(7, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/6anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/6anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/6anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38511,)
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9648,)
Epoch 1/20
1204/1204 [==============================] - 40s 32ms/step - loss: 4.6164 - accuracy: 0.8365 - val_loss: 1.4840 - val_accuracy: 0.8826
Epoch 2/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.1812 - accuracy: 0.9652 - val_loss: 0.2462 - val_accuracy: 0.9438
Epoch 3/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0727 - accuracy: 0.9772 - val_loss: 0.1626 - val_accuracy: 0.9554
Epoch 4/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0634 - accuracy: 0.9813 - val_loss: 0.1164 - val_accuracy: 0.9655
Epoch 5/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0561 - accuracy: 0.9836 - val_loss: 0.1070 - val_accuracy: 0.9663
Epoch 6/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0515 - accuracy: 0.9840 -

In [ ]:
class_0 = model0.load_model("final_models/6anomaly-classifier0")
class_1 = model1.load_model("final_models/6anomaly-classifier1")
class_2 = model2.load_model("final_models/6anomaly-classifier2")
class_3 = model3.load_model("final_models/6anomaly-classifier3")
class_4 = model4.load_model("final_models/6anomaly-classifier4")
class_5 = model5.load_model("final_models/6anomaly-classifier5")
class_6 = model6.load_model("final_models/6anomaly-classifier6")
class_7 = model7.load_model("final_models/6anomaly-classifier7")
class_8 = model8.load_model("final_models/6anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9648,)
Temperature Initial value: 1.0
Temperature Final value: 2.3399412631988525
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9501,)
Temperature Initial value: 1.0
Temperature Final value: 2.3231940269470215
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9649,)
Temperature Initial value: 1.0
Temperature Final value: 2.2757489681243896
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9604,)
Temperature Initial value: 1.0
Temperature Final value: 2.1671793460845947
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9647,)
Temperature Initial value: 1.0
Temperature Final value: 2.4774105548858643
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 7: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9719,)
Temperature Initial value: 1.0
Temperature Final value: 2.0073540210723877
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 6, 9: 7}
{0, 1, 2,

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3399413>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.323194>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.275749>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1671793>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4774106>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.007354>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.019063>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4026806>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4807267>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10823it [12:28, 14.46it/s]

[9420.375517904758, 9262.270715504885, 9523.519088238478, 9475.3318811059, 9399.644717305899, 9602.747386485338, 9385.62716987729, 9502.379462838173, 9273.205349057913]
[9648, 9501, 9649, 9604, 9647, 9719, 9524, 9663, 9629]
[703.5884800964678, 738.8165594201257, 407.5849095720339, 424.0141194273701, 738.1703139624831, 374.3955660142659, 455.14935245186825, 482.94503568533435, 1072.3461702701527]
Entropy: 0.0623615202184385
Each Classifier Average:  [0.9764070810431964, 0.9748732465535086, 0.9869954490867943, 0.9866026531763744, 0.9743593570338861, 0.9880386239824404, 0.9854711434142472, 0.9833777773815764, 0.9630496779580343]
Threshold: 0.9799083344033398


In [ ]:
ref_vect_in_6an = []
ref_vect_in_6an.append(classifier_avg_t)

threshold_in_6an = []
threshold_in_6an.append(treshold_t)

entropy_in_6an = []
entropy_in_6an.append(entropy_t)

print(entropy_in_6an)
print(ref_vect_in_6an)
print(threshold_in_6an)

[0.0623615202184385]
[[0.9764070810431964, 0.9748732465535086, 0.9869954490867943, 0.9866026531763744, 0.9743593570338861, 0.9880386239824404, 0.9854711434142472, 0.9833777773815764, 0.9630496779580343]]
[0.9799083344033398]


In [ ]:
entropy_in_6an = [0.0623615202184385]
ref_vect_in_6an = [[0.9764070810431964, 0.9748732465535086, 0.9869954490867943, 0.9866026531763744, 0.9743593570338861, 0.9880386239824404, 0.9854711434142472, 0.9833777773815764, 0.9630496779580343]]
threshold_in_6an = [0.9799083344033398]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 7:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10823it [01:34, 114.24it/s]

[810.8496254086494, 771.8726341128349, 922.2594998776913, 1027.8617151379585, 915.4727564007044, 899.0839091837406, 900.8293139487505, 910.9361905455589, 1010.5103847384453]
[1175, 1322, 1174, 1219, 1176, 1104, 1299, 1160, 1194]
[959.1896780845709, 1370.0300924649928, 652.1723656872637, 519.1910479807548, 676.2272188120842, 524.7716135883747, 1020.1400757234369, 667.1080591225764, 439.6067816337766]
Entropy: 0.6236724091756243
Each Classifier Average:  [0.6900847875818293, 0.5838673480429917, 0.7855702724682209, 0.8432007507284319, 0.7784632282318915, 0.8143875988983158, 0.6934790715540804, 0.7852898194358267, 0.8463236053085806]
Threshold: 0.7578518313611299


In [ ]:
ref_vect_out_6an = []
ref_vect_out_6an.append(classifier_avg_t)

threshold_out_6an = []
threshold_out_6an.append(treshold_t)

entropy_out_6an = []
entropy_out_6an.append(entropy_t)

print(entropy_out_6an)
print(ref_vect_out_6an)
print(threshold_out_6an)

[0.6236724091756243]
[[0.6900847875818293, 0.5838673480429917, 0.7855702724682209, 0.8432007507284319, 0.7784632282318915, 0.8143875988983158, 0.6934790715540804, 0.7852898194358267, 0.8463236053085806]]
[0.7578518313611299]


In [ ]:
entropy_out_6an = [0.6236724091756243]
ref_vect_out_6an = [[0.6900847875818293, 0.5838673480429917, 0.7855702724682209, 0.8432007507284319, 0.7784632282318915, 0.8143875988983158, 0.6934790715540804, 0.7852898194358267, 0.8463236053085806]]
threshold_out_6an = [0.7578518313611299]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [ ]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_6an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 6:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 6)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 6)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 6)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 6)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(4, 6)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(5, 6)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(7, 6)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 6)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 6)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_6an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:39,  8.07it/s]


In [ ]:
pickle_out = open("sm_all_wt_6an.pickle","wb")
pickle.dump(max_sm_all_wt_6an, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_6an.pickle","rb")
max_sm_all_wt_6an = pickle.load(pickle_in)

In [ ]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_6an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 2, 7, 7] 1 7


In [ ]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_6an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_6an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_6an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_6an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_6an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in > sim_ood:
    y_ = ind
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
#  if lbl == 1:
#    y_true.append(0)
#  else:
#    y_true.append(1)
  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_6an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.8977
Accuracy of determining ID data:  0.8969254589692546
Accuracy of determining OOD data:  0.9050104384133612


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9164895761325367

In [ ]:
roc_auc_score(y_true, y_pred)

0.8888992403347127

In [ ]:
sim_score_ = sim_score_ / max(sim_score_)

In [ ]:
roc_auc_score(y_true, sim_score_)

0.7098903793431627

In [ ]:
sim_score_

array([0.96736581, 0.98604915, 0.94387278, ..., 0.97899977, 0.97265618,
       0.9870728 ])

# 7 as anomaly

In [11]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(7, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='7anomaly:classifier0')
model1 = Model(name='7anomaly:classifier1')
model2 = Model(name='7anomaly:classifier2')
model3 = Model(name='7anomaly:classifier3')
model4 = Model(name='7anomaly:classifier4')
model5 = Model(name='7anomaly:classifier5')
model6 = Model(name='7anomaly:classifier6')
model7 = Model(name='7anomaly:classifier7')
model8 = Model(name='7anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/7anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/7anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/7anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/7anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(4, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/7anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(5, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(5, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/7anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(6, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(6, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/7anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(8, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(8, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/7anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/7anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38286,)
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9526,)
Epoch 1/20
1197/1197 [==============================] - 40s 32ms/step - loss: 5.7006 - accuracy: 0.7752 - val_loss: 0.1483 - val_accuracy: 0.9666
Epoch 2/20
1197/1197 [==============================] - 38s 32ms/step - loss: 0.1602 - accuracy: 0.9661 - val_loss: 0.2425 - val_accuracy: 0.9596
Epoch 3/20
1197/1197 [==============================] - 38s 32ms/step - loss: 0.0924 - accuracy: 0.9749 - val_loss: 0.0798 - val_accuracy: 0.9790
Epoch 4/20
1197/1197 [==============================] - 38s 32ms/step - loss: 0.0782 - accuracy: 0.9778 - val_loss: 0.0852 - val_accuracy: 0.9785
Epoch 5/20
1197/1197 [==============================] - 38s 32ms/step - loss: 0.0626 - accuracy: 0.9821 - val_loss: 0.1244 - val_accuracy: 0.9710
Epoch 6/20
1197/1197 [==============================] - 38s 32ms/step - loss: 0.0508 - accuracy: 0.9842 -

In [ ]:
class_0 = model0.load_model("final_models/7anomaly-classifier0")
class_1 = model1.load_model("final_models/7anomaly-classifier1")
class_2 = model2.load_model("final_models/7anomaly-classifier2")
class_3 = model3.load_model("final_models/7anomaly-classifier3")
class_4 = model4.load_model("final_models/7anomaly-classifier4")
class_5 = model5.load_model("final_models/7anomaly-classifier5")
class_6 = model6.load_model("final_models/7anomaly-classifier6")
class_7 = model7.load_model("final_models/7anomaly-classifier7")
class_8 = model8.load_model("final_models/7anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_37', 'dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9526,)
Temperature Initial value: 1.0
Temperature Final value: 2.184089422225952
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9379,)
Temperature Initial value: 1.0
Temperature Final value: 2.3121402263641357
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9527,)
Temperature Initial value: 1.0
Temperature Final value: 1.8578717708587646
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9482,)
Temperature Initial value: 1.0
Temperature Final value: 2.017979383468628
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9525,)
Temperature Initial value: 1.0
Temperature Final value: 2.3343472480773926
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 8: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9597,)
Temperature Initial value: 1.0
Temperature Final value: 2.3091700077056885
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 8: 6, 9: 7}
{0, 1, 2, 3

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1840894>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3121402>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.8578718>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0179794>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3343472>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.30917>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2927048>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3133302>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1407003>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10701it [12:20, 14.45it/s]

[9429.74279871583, 9203.4357637465, 9428.05303543806, 9331.52282872796, 9374.472832918167, 9452.06161659956, 9320.431676834822, 9396.391365140676, 9365.88415876031]
[9526, 9379, 9527, 9482, 9525, 9597, 9524, 9541, 9507]
[295.5887232365815, 523.7630844539426, 308.2188068750683, 495.9309783718782, 446.70406029953773, 463.49675650828357, 622.1243507756442, 458.75812259034694, 462.52658726557945]
Entropy: 0.04764202276565664
Each Classifier Average:  [0.9898953179420354, 0.9812811348487579, 0.9896140480149114, 0.9841302287205189, 0.9841966228785477, 0.9848975322079357, 0.9786257535525852, 0.9848434509108769, 0.9851566381361427]
Threshold: 0.9847378585791459


In [ ]:
ref_vect_in_7an = []
ref_vect_in_7an.append(classifier_avg_t)

threshold_in_7an = []
threshold_in_7an.append(treshold_t)

entropy_in_7an = []
entropy_in_7an.append(entropy_t)

print(entropy_in_7an)
print(ref_vect_in_7an)
print(threshold_in_7an)

[0.04764202276565664]
[[0.9898953179420354, 0.9812811348487579, 0.9896140480149114, 0.9841302287205189, 0.9841966228785477, 0.9848975322079357, 0.9786257535525852, 0.9848434509108769, 0.9851566381361427]]
[0.9847378585791459]


In [12]:
entropy_in_7an = [0.04764202276565664]
ref_vect_in_7an = [[0.9898953179420354, 0.9812811348487579, 0.9896140480149114, 0.9841302287205189, 0.9841966228785477, 0.9848975322079357, 0.9786257535525852, 0.9848434509108769, 0.9851566381361427]]
threshold_in_7an = [0.9847378585791459]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 8:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10701it [01:33, 115.03it/s]

[901.7580592930317, 767.9853129386902, 964.5302682518959, 995.1774223446846, 1032.1835075318813, 830.8286636769772, 959.181392878294, 874.544561907649, 999.2052688896656]
[1175, 1322, 1174, 1219, 1176, 1104, 1177, 1160, 1194]
[712.4436416855642, 1541.9443767247722, 532.1211717518565, 601.0975149059086, 390.73232021798367, 716.8298604735173, 601.9435984068477, 752.3643363680603, 512.3374292557655]
Entropy: 0.5877480370363201
Each Classifier Average:  [0.7674536674834312, 0.5809268630398564, 0.8215760376932674, 0.8163883694378052, 0.8777070642277902, 0.7525621953595808, 0.8149374620886101, 0.7539177257824561, 0.8368553340784469]
Threshold: 0.7802583021323605


In [ ]:
ref_vect_out_7an = []
ref_vect_out_7an.append(classifier_avg_t)

threshold_out_7an = []
threshold_out_7an.append(treshold_t)

entropy_out_7an = []
entropy_out_7an.append(entropy_t)

print(entropy_out_7an)
print(ref_vect_out_7an)
print(threshold_out_7an)

[0.5877480370363201]
[[0.7674536674834312, 0.5809268630398564, 0.8215760376932674, 0.8163883694378052, 0.8777070642277902, 0.7525621953595808, 0.8149374620886101, 0.7539177257824561, 0.8368553340784469]]
[0.7802583021323605]


In [13]:
entropy_out_7an = [0.5877480370363201]
ref_vect_out_7an = [[0.7674536674834312, 0.5809268630398564, 0.8215760376932674, 0.8163883694378052, 0.8777070642277902, 0.7525621953595808, 0.8149374620886101, 0.7539177257824561, 0.8368553340784469]]
threshold_out_7an = [0.7802583021323605]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [14]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [15]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_7an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 7:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 7)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 7)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 7)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 7)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(4, 7)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(5, 7)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(6, 7)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(8, 7)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 7)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_7an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:25,  8.16it/s]


In [ ]:
pickle_out = open("sm_all_wt_7an.pickle","wb")
pickle.dump(max_sm_all_wt_7an, pickle_out)
pickle_out.close()

In [16]:
pickle_in = open("sm_all_wt_7an.pickle","rb")
max_sm_all_wt_7an = pickle.load(pickle_in)

In [17]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_7an:
  print(p, lbl, a_l)
  break

[9, 9, 3, 9, 1, 2, 9, 9, 1] 0 7


In [24]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_7an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_7an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_7an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_7an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_7an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
#  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood2)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)

  if sim_in >= sim_ood:
    y_ = ind
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_7an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9351
Accuracy of determining ID data:  0.9410387873383861
Accuracy of determining OOD data:  0.8832684824902723


In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.024163480503980393

In [20]:
roc_auc_score(y_true, y_pred)

0.0654066867782344

In [25]:
roc_auc_score(y_true, sim_score_)

0.9756027615530201

# 8 as anomaly

In [26]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(8, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='8anomaly:classifier0')
model1 = Model(name='8anomaly:classifier1')
model2 = Model(name='8anomaly:classifier2')
model3 = Model(name='8anomaly:classifier3')
model4 = Model(name='8anomaly:classifier4')
model5 = Model(name='8anomaly:classifier5')
model6 = Model(name='8anomaly:classifier6')
model7 = Model(name='8anomaly:classifier7')
model8 = Model(name='8anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/8anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/8anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/8anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/8anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(4, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/8anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(5, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(5, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/8anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(6, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(6, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/8anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(7, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(7, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/8anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(9, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(9, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/8anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38561,)
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9665,)
Epoch 1/20
1206/1206 [==============================] - 38s 31ms/step - loss: 5.0723 - accuracy: 0.8471 - val_loss: 1.0640 - val_accuracy: 0.8619
Epoch 2/20
1206/1206 [==============================] - 38s 31ms/step - loss: 0.3995 - accuracy: 0.9545 - val_loss: 0.1423 - val_accuracy: 0.9617
Epoch 3/20
1206/1206 [==============================] - 39s 32ms/step - loss: 0.0625 - accuracy: 0.9815 - val_loss: 0.1176 - val_accuracy: 0.9681
Epoch 4/20
1206/1206 [==============================] - 38s 32ms/step - loss: 0.0557 - accuracy: 0.9824 - val_loss: 0.1520 - val_accuracy: 0.9596
Epoch 5/20
1206/1206 [==============================] - 38s 32ms/step - loss: 0.0527 - accuracy: 0.9839 - val_loss: 0.1089 - val_accuracy: 0.9728
Epoch 6/20
1206/1206 [==============================] - 39s 32ms/step - loss: 0.0550 - accuracy: 0.9835 -

In [ ]:
class_0 = model0.load_model("final_models/8anomaly-classifier0")
class_1 = model1.load_model("final_models/8anomaly-classifier1")
class_2 = model2.load_model("final_models/8anomaly-classifier2")
class_3 = model3.load_model("final_models/8anomaly-classifier3")
class_4 = model4.load_model("final_models/8anomaly-classifier4")
class_5 = model5.load_model("final_models/8anomaly-classifier5")
class_6 = model6.load_model("final_models/8anomaly-classifier6")
class_7 = model7.load_model("final_models/8anomaly-classifier7")
class_8 = model8.load_model("final_models/8anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_19', 'dense_21', 'dense_23', 'dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9665,)
Temperature Initial value: 1.0
Temperature Final value: 2.289517879486084
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9518,)
Temperature Initial value: 1.0
Temperature Final value: 2.1803793907165527
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9666,)
Temperature Initial value: 1.0
Temperature Final value: 1.9876495599746704
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9621,)
Temperature Initial value: 1.0
Temperature Final value: 1.944435954093933
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9664,)
Temperature Initial value: 1.0
Temperature Final value: 2.4403905868530273
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 9: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9736,)
Temperature Initial value: 1.0
Temperature Final value: 1.9343997240066528
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 7: 6, 9: 7}
{0, 1, 2, 3

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2895179>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.1803794>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9876496>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.944436>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.4403906>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9343997>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2423983>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2220883>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2203646>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 7:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10840it [12:30, 14.45it/s]

[9510.209820628166, 9385.876577079296, 9520.956473916769, 9422.133226245642, 9467.75987970829, 9608.657942205667, 9525.557143509388, 9388.458064109087, 9450.711543440819]
[9665, 9518, 9666, 9621, 9664, 9736, 9663, 9541, 9646]
[481.30395679948793, 394.72724081672425, 464.1030757331269, 630.9795363556774, 564.8434673613069, 400.83978233250735, 470.2301088136472, 439.5882373143666, 597.977337849038]
Entropy: 0.051246196019809176
Each Classifier Average:  [0.9839844615238661, 0.986118572922809, 0.9849944624370752, 0.979329926852265, 0.9796936961618677, 0.9869204952963915, 0.9857763782996366, 0.984011955152404, 0.9797544623098506]
Threshold: 0.9833982678840185


In [ ]:
ref_vect_in_8an = []
ref_vect_in_8an.append(classifier_avg_t)

threshold_in_8an = []
threshold_in_8an.append(treshold_t)

entropy_in_8an = []
entropy_in_8an.append(entropy_t)

print(entropy_in_8an)
print(ref_vect_in_8an)
print(threshold_in_8an)

[0.051246196019809176]
[[0.9839844615238661, 0.986118572922809, 0.9849944624370752, 0.979329926852265, 0.9796936961618677, 0.9869204952963915, 0.9857763782996366, 0.984011955152404, 0.9797544623098506]]
[0.9833982678840185]


In [27]:
entropy_in_8an = [0.051246196019809176]
ref_vect_in_8an = [[0.9839844615238661, 0.986118572922809, 0.9849944624370752, 0.979329926852265, 0.9796936961618677, 0.9869204952963915, 0.9857763782996366, 0.984011955152404, 0.9797544623098506]]
threshold_in_8an = [0.9833982678840185]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 7:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 9:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10840it [01:33, 115.56it/s]

[988.5373156964779, 1055.8952522277832, 925.3744595348835, 993.735117405653, 1043.13758456707, 892.3960430324078, 1017.8893555998802, 993.6414393186569, 1076.8205579817295]
[1175, 1322, 1174, 1219, 1176, 1104, 1177, 1299, 1194]
[483.3240665173653, 854.2080204254016, 652.735131484762, 613.2850650179328, 351.530580905099, 543.6085416395326, 407.89182854065996, 781.7942911342107, 296.23225119417293]
Entropy: 0.45604455692879053
Each Classifier Average:  [0.8413083537842365, 0.7987104782358421, 0.7882235600808207, 0.8152051824492641, 0.8870217555842432, 0.8083297491235577, 0.8648167847067801, 0.7649279748411524, 0.9018597638037935]
Threshold: 0.83004484473441


In [ ]:
ref_vect_out_8an = []
ref_vect_out_8an.append(classifier_avg_t)

threshold_out_8an = []
threshold_out_8an.append(treshold_t)

entropy_out_8an = []
entropy_out_8an.append(entropy_t)

print(entropy_out_8an)
print(ref_vect_out_8an)
print(threshold_out_8an)

[0.45604455692879053]
[[0.8413083537842365, 0.7987104782358421, 0.7882235600808207, 0.8152051824492641, 0.8870217555842432, 0.8083297491235577, 0.8648167847067801, 0.7649279748411524, 0.9018597638037935]]
[0.83004484473441]


In [28]:
entropy_out_8an = [0.45604455692879053]
ref_vect_out_8an = [[0.8413083537842365, 0.7987104782358421, 0.7882235600808207, 0.8152051824492641, 0.8870217555842432, 0.8083297491235577, 0.8648167847067801, 0.7649279748411524, 0.9018597638037935]]
threshold_out_8an = [0.83004484473441]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [29]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_8an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 8:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 8)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 8)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 8)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 8)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(4, 8)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(5, 8)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(6, 8)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(7, 8)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(9, 8)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_8an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:34,  8.10it/s]


In [ ]:
pickle_out = open("sm_all_wt_8an.pickle","wb")
pickle.dump(max_sm_all_wt_8an, pickle_out)
pickle_out.close()

In [30]:
pickle_in = open("sm_all_wt_8an.pickle","rb")
max_sm_all_wt_8an = pickle.load(pickle_in)

In [31]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_8an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 7, 9, 7] 1 7


In [36]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_8an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_8an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_8an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_8an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_8an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
#  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in >= sim_ood:
    y_ = ind
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_8an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9338
Accuracy of determining ID data:  0.934190117438511
Accuracy of determining OOD data:  0.9301848049281314


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9692839212842115

In [ ]:
roc_auc_score(y_true, y_pred)

0.9351349125569709

In [37]:
roc_auc_score(y_true, sim_score_)

0.9124137615676545

# 9 as anomaly

In [38]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(9, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='9anomaly:classifier0')
model1 = Model(name='9anomaly:classifier1')
model2 = Model(name='9anomaly:classifier2')
model3 = Model(name='9anomaly:classifier3')
model4 = Model(name='9anomaly:classifier4')
model5 = Model(name='9anomaly:classifier5')
model6 = Model(name='9anomaly:classifier6')
model7 = Model(name='9anomaly:classifier7')
model8 = Model(name='9anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
train_imgs0, train_lbls0 = get_train_data(0, new_train_images, new_train_labels)
val_imgs0, val_lbls0 = get_train_data(0, new_val_images, new_val_labels)
class_0 = model0.build()
model0.train2(train_imgs0, train_lbls0, val_imgs0, val_lbls0)
#model0.train(train_data0)
model0.save_model("final_models/9anomaly-classifier0")
print('Model 0 saved')

train_imgs1, train_lbls1 = get_train_data(1, new_train_images, new_train_labels)
val_imgs1, val_lbls1 = get_train_data(1, new_val_images, new_val_labels)
class_1 = model1.build()
model1.train2(train_imgs1, train_lbls1, val_imgs1, val_lbls1)
#model1.train(train_data1)
model1.save_model("final_models/9anomaly-classifier1")
print('Model 1 saved')

#train_data2 = get_train_data(2, new_train_images)
train_imgs2, train_lbls2 = get_train_data(2, new_train_images, new_train_labels)
val_imgs2, val_lbls2 = get_train_data(2, new_val_images, new_val_labels)
class_2 = model2.build()
model2.train2(train_imgs2, train_lbls2, val_imgs2, val_lbls2)
#model2.train(train_data2)
model2.save_model("final_models/9anomaly-classifier2")
print('Model 2 saved')

#train_data3 = get_train_data(3, new_train_images)
train_imgs3, train_lbls3 = get_train_data(3, new_train_images, new_train_labels)
val_imgs3, val_lbls3 = get_train_data(3, new_val_images, new_val_labels)
class_3 = model3.build()
model3.train2(train_imgs3, train_lbls3, val_imgs3, val_lbls3)
#model3.train(train_data3)
model3.save_model("final_models/9anomaly-classifier3")
print('Model 3 saved')

#train_data4 = get_train_data(4, new_train_images)
train_imgs4, train_lbls4 = get_train_data(4, new_train_images, new_train_labels)
val_imgs4, val_lbls4 = get_train_data(4, new_val_images, new_val_labels)
class_4 = model4.build()
model4.train2(train_imgs4, train_lbls4, val_imgs4, val_lbls4)
#model4.train(train_data4)
model4.save_model("final_models/9anomaly-classifier4")
print('Model 4 saved')

#train_data5 = get_train_data(5, new_train_images)
train_imgs5, train_lbls5 = get_train_data(5, new_train_images, new_train_labels)
val_imgs5, val_lbls5 = get_train_data(5, new_val_images, new_val_labels)
class_5 = model5.build()
model5.train2(train_imgs5, train_lbls5, val_imgs5, val_lbls5)
#model5.train(train_data5)
model5.save_model("final_models/9anomaly-classifier5")
print('Model 5 saved')

#train_data6 = get_train_data(6, new_train_images)
train_imgs6, train_lbls6 = get_train_data(6, new_train_images, new_train_labels)
val_imgs6, val_lbls6 = get_train_data(6, new_val_images, new_val_labels)
class_6 = model6.build()
model6.train2(train_imgs6, train_lbls6, val_imgs6, val_lbls6)
#model6.train(train_data6)
model6.save_model("final_models/9anomaly-classifier6")
print('Model 6 saved')

#train_data7 = get_train_data(7, new_train_images)
train_imgs7, train_lbls7 = get_train_data(7, new_train_images, new_train_labels)
val_imgs7, val_lbls7 = get_train_data(7, new_val_images, new_val_labels)
class_7 = model7.build()
#model7.train(train_data7)
model7.train2(train_imgs7, train_lbls7, val_imgs7, val_lbls7)
model7.save_model("final_models/9anomaly-classifier7")
print('Model 7 saved')

#train_data8 = get_train_data(8, new_train_images)
train_imgs8, train_lbls8 = get_train_data(8, new_train_images, new_train_labels)
val_imgs8, val_lbls8 = get_train_data(8, new_val_images, new_val_labels)
class_8 = model8.build()
#model8.train(train_data8)
model8.train2(train_imgs8, train_lbls8, val_imgs8, val_lbls8)
model8.save_model("final_models/9anomaly-classifier8")
print('Model 8 saved')

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(38497,)
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9631,)
Epoch 1/20
1204/1204 [==============================] - 40s 32ms/step - loss: 4.8472 - accuracy: 0.8528 - val_loss: 0.2174 - val_accuracy: 0.9646
Epoch 2/20
1204/1204 [==============================] - 39s 32ms/step - loss: 0.1497 - accuracy: 0.9721 - val_loss: 0.2839 - val_accuracy: 0.9133
Epoch 3/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0644 - accuracy: 0.9818 - val_loss: 0.1244 - val_accuracy: 0.9639
Epoch 4/20
1204/1204 [==============================] - 39s 32ms/step - loss: 0.0487 - accuracy: 0.9864 - val_loss: 0.1694 - val_accuracy: 0.9537
Epoch 5/20
1204/1204 [==============================] - 38s 32ms/step - loss: 0.0549 - accuracy: 0.9837 - val_loss: 0.1745 - val_accuracy: 0.9572
Epoch 6/20
1204/1204 [==============================] - 39s 32ms/step - loss: 0.0489 - accuracy: 0.9853 -

In [ ]:
class_0 = model0.load_model("final_models/9anomaly-classifier0")
class_1 = model1.load_model("final_models/9anomaly-classifier1")
class_2 = model2.load_model("final_models/9anomaly-classifier2")
class_3 = model3.load_model("final_models/9anomaly-classifier3")
class_4 = model4.load_model("final_models/9anomaly-classifier4")
class_5 = model5.load_model("final_models/9anomaly-classifier5")
class_6 = model6.load_model("final_models/9anomaly-classifier6")
class_7 = model7.load_model("final_models/9anomaly-classifier7")
class_8 = model8.load_model("final_models/9anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_37', 'dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8


In [ ]:
temp_val = []
y_pred = []
labels = set(new_val_labels)
for i, lo in zip(range(9), labels):
  m = model_dict[i]
  val_imgs_, val_lbls_ = get_train_data(lo, new_val_images, new_val_labels)
  y_p = m.predict(val_imgs_)
  y_pred.append(y_p)
  temp = temp_cal(y_p, val_lbls_)
  temp_val.append(temp)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9631,)
Temperature Initial value: 1.0
Temperature Final value: 2.398526191711426
{0: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9484,)
Temperature Initial value: 1.0
Temperature Final value: 2.31925106048584
{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9632,)
Temperature Initial value: 1.0
Temperature Final value: 2.21435546875
{0: 0, 1: 1, 2: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9587,)
Temperature Initial value: 1.0
Temperature Final value: 1.9258661270141602
{0: 0, 1: 1, 2: 2, 3: 3, 5: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9630,)
Temperature Initial value: 1.0
Temperature Final value: 2.2611944675445557
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5, 6, 7}
(9702,)
Temperature Initial value: 1.0
Temperature Final value: 2.222395420074463
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 7: 6, 8: 7}
{0, 1, 2, 3, 4, 5,

In [ ]:
temp_val

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3985262>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.319251>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2143555>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.9258661>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2611945>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2223954>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.3583956>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2993236>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0704982>]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]
for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if not lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if not lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if not lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if not lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if not lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if not lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if not lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if not lbl == 7:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if not lbl == 8:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10806it [12:16, 14.67it/s]

[9495.856197595596, 9330.759849369526, 9472.157935276628, 9489.994508206844, 9441.329641819, 9609.918594390154, 9452.439297020435, 9403.818867236376, 9527.683840841055]
[9631, 9484, 9632, 9587, 9630, 9702, 9629, 9507, 9646]
[414.3286874832835, 468.35928304922413, 509.49605015605584, 318.3835482986842, 564.8503037065246, 298.0027289714437, 540.6929349365489, 329.0205501542547, 365.18769117814077]
Entropy: 0.04405569144510324
Each Classifier Average:  [0.9859678327894918, 0.9838422447669259, 0.9834051012538026, 0.9898815592163184, 0.9804080624941849, 0.9905090284879565, 0.9816636511600826, 0.9891468252063086, 0.9877341738379696]
Threshold: 0.9858398310236711


In [ ]:
ref_vect_in_9an = []
ref_vect_in_9an.append(classifier_avg_t)

threshold_in_9an = []
threshold_in_9an.append(treshold_t)

entropy_in_9an = []
entropy_in_9an.append(entropy_t)

print(entropy_in_9an)
print(ref_vect_in_9an)
print(threshold_in_9an)

[0.04405569144510324]
[[0.9859678327894918, 0.9838422447669259, 0.9834051012538026, 0.9898815592163184, 0.9804080624941849, 0.9905090284879565, 0.9816636511600826, 0.9891468252063086, 0.9877341738379696]]
[0.9858398310236711]


In [39]:
entropy_in_9an = [0.04405569144510324]
ref_vect_in_9an = [[0.9859678327894918, 0.9838422447669259, 0.9834051012538026, 0.9898815592163184, 0.9804080624941849, 0.9905090284879565, 0.9816636511600826, 0.9891468252063086, 0.9877341738379696]]
threshold_in_9an = [0.9858398310236711]

In [ ]:
from tqdm import tqdm
avg = [0,0,0,0,0,0,0,0,0]
count = [0,0,0,0,0,0,0,0,0]
entropy_list = [0,0,0,0,0,0,0,0,0]

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  if lbl == 0:
    logits = model_dict[0](img)
    logits = tf.math.divide(logits, temp_val[0])
    pred = tf.nn.softmax(logits)
    e0 = entropy(pred[0])
    entropy_list[0] = entropy_list[0] + e0
    max_val0 = np.max(pred)
    avg[0] = avg[0] + max_val0
    count[0] = count[0] + 1
  if lbl == 1:
    logits = model_dict[1](img)
    logits = tf.math.divide(logits, temp_val[1])
    pred = tf.nn.softmax(logits)
    e1 = entropy(pred[0])
    entropy_list[1] = entropy_list[1] + e1
    max_val1 = np.max(pred)
    avg[1] = avg[1] + max_val1
    count[1] = count[1] + 1
  if lbl == 2:
    logits = model_dict[2](img)
    logits = tf.math.divide(logits, temp_val[2])
    pred = tf.nn.softmax(logits)
    e2 = entropy(pred[0])
    entropy_list[2] = entropy_list[2] + e2
    max_val2 = np.max(pred)
    avg[2] = avg[2] + max_val2
    count[2] = count[2] + 1
  if lbl == 3:
    logits = model_dict[3](img)
    logits = tf.math.divide(logits, temp_val[3])
    pred = tf.nn.softmax(logits)
    e3 = entropy(pred[0])
    entropy_list[3] = entropy_list[3] + e3
    max_val3 = np.max(pred)  
    avg[3] = avg[3] + max_val3
    count[3] = count[3] + 1
  if lbl == 4:
    logits = model_dict[4](img)
    logits = tf.math.divide(logits, temp_val[4])
    pred = tf.nn.softmax(logits)
    e4 = entropy(pred[0])
    entropy_list[4] = entropy_list[4] + e4
    max_val4 = np.max(pred)
    avg[4] = avg[4] + max_val4
    count[4] = count[4] + 1
  if lbl == 5:
    logits = model_dict[5](img)
    logits = tf.math.divide(logits, temp_val[5])
    pred = tf.nn.softmax(logits)
    e5 = entropy(pred[0])
    entropy_list[5] = entropy_list[5] + e5
    max_val5 = np.max(pred)
    avg[5] = avg[5] + max_val5
    count[5] = count[5] + 1
  if lbl == 6:
    logits = model_dict[6](img)
    logits = tf.math.divide(logits, temp_val[6])
    pred = tf.nn.softmax(logits)
    e6 = entropy(pred[0])
    entropy_list[6] = entropy_list[6] + e6
    max_val6 = np.max(pred)
    avg[6] = avg[6] + max_val6
    count[6] = count[6] + 1
  if lbl == 7:
    logits = model_dict[7](img)
    logits = tf.math.divide(logits, temp_val[7])
    pred = tf.nn.softmax(logits)
    e7 = entropy(pred[0])
    entropy_list[7] = entropy_list[7] + e7
    max_val7 = np.max(pred)
    avg[7] = avg[7] + max_val7
    count[7] = count[7] + 1
  if lbl == 8:
    logits = model_dict[8](img)
    logits = tf.math.divide(logits, temp_val[8])
    pred = tf.nn.softmax(logits)
    e8 = entropy(pred[0])
    entropy_list[8] = entropy_list[8] + e8
    max_val8 = np.max(pred)
    avg[8] = avg[8] + max_val8
    count[8] = count[8] + 1

print(avg)
print(count)
print(entropy_list)

entropy_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  entropy_avg_t[i] = entropy_list[i]/count[i]

entropy_value_t = 0.0
for i in range(9):
  entropy_value_t = entropy_value_t + entropy_avg_t[i]

entropy_t = entropy_value_t/len(entropy_avg_t)

print('Entropy:', entropy_t)

classifier_avg_t = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
for i in range(9):
  classifier_avg_t[i] = avg[i]/count[i]

print('Each Classifier Average: ',classifier_avg_t)

treshold_value = 0.0
for i in range(9):
  treshold_value = treshold_value + classifier_avg_t[i]

treshold_t = treshold_value/len(classifier_avg_t)

print('Threshold:', treshold_t)

10806it [01:32, 117.01it/s]

[953.0528710186481, 707.4310759902, 915.7182317972183, 1054.7954660952091, 889.6830076575279, 905.4954099357128, 943.0036944448948, 967.1280114203691, 867.1922775655985]
[1175, 1322, 1174, 1219, 1176, 1104, 1177, 1299, 1160]
[568.9805146824801, 1590.943137747934, 676.4559725443905, 408.4250742853842, 723.1622181375569, 510.1251802155093, 608.5655140470835, 877.3564862636849, 777.0138058165458]
Entropy: 0.615358066135967
Each Classifier Average:  [0.8111088263988495, 0.5351218426552194, 0.7799984938647515, 0.8652957063947573, 0.7565331697768095, 0.820195117695392, 0.8011926036065377, 0.7445173298078285, 0.747579549625516]
Threshold: 0.7623936266472957


In [ ]:
ref_vect_out_9an = []
ref_vect_out_9an.append(classifier_avg_t)

threshold_out_9an = []
threshold_out_9an.append(treshold_t)

entropy_out_9an = []
entropy_out_9an.append(entropy_t)

print(entropy_out_9an)
print(ref_vect_out_9an)
print(threshold_out_9an)

[0.615358066135967]
[[0.8111088263988495, 0.5351218426552194, 0.7799984938647515, 0.8652957063947573, 0.7565331697768095, 0.820195117695392, 0.8011926036065377, 0.7445173298078285, 0.747579549625516]]
[0.7623936266472957]


In [40]:
entropy_out_9an = [0.615358066135967]
ref_vect_out_9an = [[0.8111088263988495, 0.5351218426552194, 0.7799984938647515, 0.8652957063947573, 0.7565331697768095, 0.820195117695392, 0.8011926036065377, 0.7445173298078285, 0.747579549625516]]
threshold_out_9an = [0.7623936266472957]

In [ ]:
def get_max_pred_value(model, img, temp):
  logits = model(img)
  logits = tf.math.divide(logits, temp)
  pred = tf.nn.softmax(logits)
  e = entropy(pred[0])

  return np.max(pred), tf.argmax(pred[0]).numpy(), e

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


In [41]:
ood = 0
ind = 1

In [ ]:
max_sm_all_wt_9an = []
for data, label in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  ref_value = []
  
  if label == 9:
    lbl = ood
  else:
    lbl = ind


  max0, pred_0, e0 = get_max_pred_value(model_dict[0], img, temp=temp_val[0])
  map0 = get_mapping(0, 9)
  pred_0 = map0[pred_0]

  max1, pred_1, e1 = get_max_pred_value(model_dict[1], img, temp=temp_val[1])
  map1 = get_mapping(1, 9)
  pred_1 = map1[pred_1]

  max2, pred_2, e2 = get_max_pred_value(model_dict[2], img, temp=temp_val[2])
  map2 = get_mapping(2, 9)
  pred_2 = map2[pred_2]

  max3, pred_3, e3 = get_max_pred_value(model_dict[3], img, temp=temp_val[3])
  map3 = get_mapping(3, 9)
  pred_3 = map3[pred_3]

  max4, pred_4, e4 = get_max_pred_value(model_dict[4], img, temp=temp_val[4])
  map4 = get_mapping(4, 9)
  pred_4 = map4[pred_4]

  max5, pred_5, e5 = get_max_pred_value(model_dict[5], img, temp=temp_val[5])
  map5 = get_mapping(5, 9)
  pred_5 = map5[pred_5]

  max6, pred_6, e6 = get_max_pred_value(model_dict[6], img, temp=temp_val[6])
  map6 = get_mapping(6, 9)
  pred_6 = map6[pred_6]

  max7, pred_7, e7 = get_max_pred_value(model_dict[7], img, temp=temp_val[7])
  map7 = get_mapping(7, 9)
  pred_7 = map7[pred_7]

  max8, pred_8, e8 = get_max_pred_value(model_dict[8], img, temp=temp_val[8])
  map8 = get_mapping(8, 9)
  pred_8 = map8[pred_8]

  ref_value =[max0, max1, max2, max3, max4, max5, max6, max7, max8]
  pred_value = [pred_0, pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8]
  e_value = [e0, e1, e2, e3, e4, e5, e6, e7, e8]
  e_ = 0.0
  for  e in e_value:
    e_ = e_ + e
  e_ = e_/len(e_value)
  max_sm_all_wt_9an.append([ref_value, lbl, pred_value, label, e_])


10000it [20:15,  8.23it/s]


In [ ]:
pickle_out = open("sm_all_wt_9an.pickle","wb")
pickle.dump(max_sm_all_wt_9an, pickle_out)
pickle_out.close()

In [42]:
pickle_in = open("sm_all_wt_9an.pickle","rb")
max_sm_all_wt_9an = pickle.load(pickle_in)

In [43]:
for data, lbl, p, a_l, e_ in max_sm_all_wt_9an:
  print(p, lbl, a_l)
  break

[7, 7, 7, 7, 7, 7, 7, 3, 7] 1 7


In [48]:
accuracy = 0.0
count_in = 0
count_ood = 0
acc_in = 0.0
acc_ood = 0.0
tp = 0.0
tn = 0.0
fp = 0.0
fn = 0.0
y_true = []
y_pred = []
sim_score = []
sim_score_ = []

for data, lbl, p, a_l, e_ in max_sm_all_wt_9an:

  score = 0.0
  for d in data:
    score = score + d
  score = score / len(data)
  sim_score.append(score)

  dist_ood = np.linalg.norm(np.array(data) - np.array(ref_vect_out_9an[0])) 
  dist_in = np.linalg.norm(np.array(data) - np.array(ref_vect_in_9an[0]))

  dist_ood2 = np.linalg.norm(np.array(e_) - np.array(entropy_out_9an[0])) 
  dist_in2 = np.linalg.norm(np.array(e_) - np.array(entropy_in_9an[0]))

  dist_in = dist_in / (dist_in + dist_ood)
  dist_ood = dist_ood / (dist_in + dist_ood)

  dist_in2 = dist_in2 / (dist_in2 + dist_ood2)
  dist_ood2 = dist_ood2 / (dist_in2 + dist_ood2)

  sim_in = 1 / (1 + dist_in)
  sim_ood = 1 / (1 + dist_ood)

  sim_in2 = 1 / (1 + dist_in2)
  sim_ood2 = 1 / (1 + dist_ood2)

  sim_in = sim_in + sim_in2
#  sim_ood = sim_ood + sim_ood2

  sim_score_.append(sim_ood)

  values, counts = np.unique(p, return_counts=True)
#  print(values, counts)
  #print(sim_in, sim_ood, lbl, e_)


  if sim_in >= sim_ood:
    y_ = ind
    if np.max(counts) >= 8:
      y_ = ind
    else:
      y_ = ood
  else:
    y_ = ood


  y_pred.append(y_)
  if lbl == 1:
    y_true.append(0)
  else:
    y_true.append(1)
#  y_true.append(lbl)


  if lbl == ind:
    count_in = count_in + 1
  else:
    count_ood = count_ood + 1

  if y_ == lbl:
    #sim_score_.append(1)
    accuracy = accuracy + 1
    if y_ == ind:
      acc_in = acc_in + 1
      tp = tp + 1
    else:
      acc_ood = acc_ood + 1
      tn = tn + 1
  else:
    #sim_score_.append(0)
    if y_ == ind:
      fp = fp + 1
    else:
      fn = fn + 1


print("Total Accuracy: ", accuracy/len(max_sm_all_wt_9an)) 
print("Accuracy of determining ID data: ", acc_in/count_in)
print("Accuracy of determining OOD data: ", acc_ood/count_ood)


Total Accuracy:  0.9279
Accuracy of determining ID data:  0.9470581692803915
Accuracy of determining OOD data:  0.757185332011893


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, sim_score)

0.9466390738277095

In [ ]:
roc_auc_score(y_true, y_pred)

0.8824382140096267

In [49]:
roc_auc_score(y_true, sim_score_)

0.9166690090597148